In [1]:
import sys
import os
sys.path.insert(0,os.path.abspath('..'))
from spatial_graphs.AmiraSpatialGraph import AmiraSpatialGraph,MatchBarrels
from spatial_graphs.Landmarks import Landmarks
from spatial_graphs.Surfaces import Surface
from spatial_graphs.Vectors import Vectors
from spatial_graphs.Alignment import Alignment
from spatial_graphs.Stats import Stats
from dask import compute,multiprocessing,delayed
import pathlib
import shutil
import glob
import pandas as pd
import vtk
from scipy.spatial import distance
import numpy as np
from scipy.interpolate import UnivariateSpline
from sklearn.cluster import k_means
import SimpleITK as sitk
from scipy.spatial import distance_matrix

In [2]:
input_path = '/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Original_Data/'
ref_neun_depth_profile = '/nas1/Data_Mythreya/MotorCortexProject/V10/lit_review/densities_zprofiles_averages.xlsx'
surface_resolutions = [50,100,200,300,400,450,500]
exp_names = ['MG49_lhs','MG49_rhs','MG50_lhs','MG50_rhs','MG48_lhs','MG48_rhs']
exp_names_tangential = ['MG49_lhs','MG49_rhs','MG50_lhs','MG50_rhs']
#alignment_types = ['Using_vM1_PCA','Using_vM1','Using_vS1','Using_vS1_vM1_Surface_Points','Using_Pia_WM_Surfaces']
#alignment_types = ['Using_vS1_vM1_Surface_Points']
selected_surface_resolution = 50
selected_voxel_size=400
selected_cutoff = 0.1

selected_global_alignment_method = 'Using_Local_Rabies_Surfaces'
selected_vM1_alignment_method = 'Using_Rabies_Surface'
selected_vS1_alignment_method = 'Using_Rabies_Surface'
selected_barrel_alignment_method = 'Using_Barrel_and_Projections'

In [3]:
# create output folder structure
HOME = str(pathlib.Path(input_path).parent)
output_root = HOME + '/Outputs/'
pathlib.Path((output_root)).mkdir(exist_ok=True)

output_path_spatial_graphs = output_root + '/Spatial_Graphs/'
output_path_landmarks = output_root + '/Landmarks/'
output_path_surfaces = output_root + '/Surfaces/'
output_path_axis_fileds = output_root +'/Axis_Fields/'
output_path_vM1 = output_root + '/vM1_Ref_Surfaces/'
output_path_vS1 = output_root + '/vS1_Ref_Surfaces/'
#output_path_vM1_cleaned = output_root + '/vM1_Ref_Surfaces_Cleaned/'
#output_path_vS1_cleaned = output_root + '/vS1_Ref_Surfaces_Cleaned/'
output_path_vM1_axis_field = output_root + '/vM1_Ref_Surfaces_Axis_Fields/'
output_path_vS1_axis_field = output_root + '/vS1_Ref_Surfaces_Axis_Fields/'
#output_path_vM1_axis_field_cleaned = output_root + '/vM1_Ref_Surfaces_Axis_Fields_Cleaned/'
#output_path_vS1_axis_field_cleaned = output_root + '/vS1_Ref_Surfaces_Axis_Fields_Cleaned/'
output_path_stats = output_root + '/Stats/'
output_path_rabies_layers = output_root + '/Layers_Rabies/'
output_path_alignment = output_root + 'Alignment/'
output_path_txmats = output_root + 'Transformation_Matrices/'
output_path_avg = output_root + 'Average/'
output_path_stats = output_root + 'Stats/' 
output_path_neun_profiles = output_root + 'NeuN_Profiles/'
output_path_rabies_profiles = output_root + 'Rabies_Profiles/'
output_path_neun_layer_borders = output_root+'Layer_Borders/'
output_path_neun_layer_borders_v2 = output_root+'Layer_Borders_V2/'


In [4]:
HOME = str(pathlib.Path(input_path).parent)
output_root = HOME + '/FINAL/'
pathlib.Path((output_root)).mkdir(exist_ok=True)

In [5]:
def get_local_avg_axis(axis_field,pts,center_pt,pia,wm,radius):
    
    nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(axis_field,pts,radius,axis_validation_distance=3000)
    nearby_axes_sg = AmiraSpatialGraph(generic_graph=True)
    unit_vecs = []
    for axis in nearby_axes:
        nearby_axes_sg.graph_data.add_edge(axis[0],axis[1])
        unit_vecs.append(Vectors().get_unit_vec(axis[0],axis[1]))
    #print(len(nearby_axes_sg.graph_data.edge_list))
    #nearby_axes_sg.write_spatial_graph(output_root+'_axis_field_selected.am')
    wm_proj_pt=[]
    avg_uv = np.array(unit_vecs).mean(axis=0)
    end_pt = (avg_uv*-20000 + center_pt)
    wm_proj_pt,depth_wm = wm.get_vector_intersection_pt\
                    (end_pt,center_pt,extrapolation_len=0)
    if len(wm_proj_pt) == 0:
        end_pt = (avg_uv*20000 + center_pt)
        wm_proj_pt,depth_wm = wm.get_vector_intersection_pt\
                    (end_pt,center_pt,extrapolation_len=0)
        
    
    pia_proj_pt=[]
    end_pt = (avg_uv*20000 + center_pt)
    pia_proj_pt,depth_pia = pia.get_vector_intersection_pt\
                    (end_pt,center_pt,extrapolation_len=0)
    if len(pia_proj_pt)==0:
        end_pt = (avg_uv*-20000 + center_pt)
        pia_proj_pt,depth_pia = pia.get_vector_intersection_pt\
                    (end_pt,center_pt,extrapolation_len=0)
    
    if len(wm_proj_pt) > 0 and len(pia_proj_pt) > 0:
        return avg_uv,[wm_proj_pt,pia_proj_pt]
    else:
        return avg_uv,None

In [6]:
def write_edge_sg(edge,output_filename):
    if edge is not None and len(edge)>0:
        sg_axis = AmiraSpatialGraph(generic_graph=True)
        sg_axis.graph_data.add_edge(edge[0],edge[1])
        sg_axis.write_spatial_graph(output_filename)

# Figure 1 - Geometric Map of 3rd Order Rabies

## A - Make grid for visualization of data

In [ ]:
# translate the final data appropriately so that it can be plotted at once

In [ ]:
# 2 rows , 2 cols - rows for animals and avg. cols for stages: raw data spread. clustered data. avg ref frame
# give a space of 20mm from each other

In [ ]:
# find offsets : the width and height of the data to be shifted (bb)
surf = Surface()
surf.append((Surface(output_root+'Final_Data/Surfaces/MG48_lhs_pia_open_bottom_voxel_size_50.vtk').surface))
surf.append((Surface(output_root+'Final_Data/Surfaces/MG48_rhs_pia_open_bottom_voxel_size_50.vtk').surface))
bb = surf.surface.GetBounds()
x_range = bb[1]-bb[0]
y_range = bb[3]-bb[2]
z_range = bb[5]-bb[4]

In [ ]:
x_range,y_range,z_range

In [ ]:
# # tranlate all data to produce grid 4x4
input_path = output_root+'Final_Data/'
output_path = output_root+'Data_Grid/'
pathlib.Path(output_path).mkdir(exist_ok=True)

row_offset = x_range + x_range*0.25
col_offset = y_range + y_range*0.5
 
row = 1
col = 1
for row in range(0,2):
    for col in range(0,2):

        txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, col*col_offset,row*row_offset,0,1]

        output_path = output_root+'Data_Grid/Row_{}_Col_{}/'.format(row+1,col+1)
        pathlib.Path(output_path).mkdir(exist_ok=True)

        aligner = Alignment()
        aligner.transform_folders(input_path,output_path,icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
        aligner.transform_folders(input_path+'/Landmarks/', output_path +'/Landmarks/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
        aligner.transform_folders(input_path+'/Average/',output_path+'/Average/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
        aligner.transform_folders(input_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)


## Combine lhs and rhs 

In [ ]:
# # tranlate all data to produce grid 4x4
mg49_txmat = [0.994035, 0.109059, 0, 0, -0.109059, 0.994035, 0, 0, 0, 0, 1, 0, 453.964, -1331.68, 0, 1]
mg50_txmat = [0.996434, -0.0843786, 0, 0, 0.0843786, 0.996434, 0, 0, 0, 0, 0.999999, 0, -2113.88, -383.051, 0, 1]

row = 0
col = 1
# for row in range(0,2):
#     for col in range(0,2):

        #txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, col*col_offset,row*row_offset,0,1]

output_path = output_root+'Data_Grid/Row_{}_Col_{}/'.format(row+1,col+1)
#pathlib.Path(output_path).mkdir(exist_ok=True)

#         aligner = Alignment()
#         aligner.transform_folders(output_path,output_path,exp_name='MG49',icp=None,txmat = mg49_txmat,hem_specific_alignment=False,apply_polydata_transform=True)
#         aligner.transform_folders(output_path+'/Landmarks/', output_path +'/Landmarks/',exp_name='MG49',icp=None,txmat = mg49_txmat,hem_specific_alignment=False,apply_polydata_transform=True)
#         #aligner.transform_folders(output_path+'/Average/',output_path+'/Average/',exp_name='MG49',icp=None,txmat = mg49_txmat,hem_specific_alignment=False,apply_polydata_transform=True)
#         #aligner.transform_folders(output_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',exp_name='MG49',icp=None,txmat = mg49_txmat,hem_specific_alignment=False,apply_polydata_transform=True)

aligner = Alignment()
aligner.transform_folders(output_path,output_path,exp_name='MG49',icp=None,txmat = mg49_txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(output_path+'/Landmarks/', output_path +'/Landmarks/',exp_name='MG49',icp=None,txmat = mg49_txmat,hem_specific_alignment=False,apply_polydata_transform=True)
#aligner.transform_folders(output_path+'/Average/',output_path+'/Average/',exp_name='MG50',icp=None,txmat = mg50_txmat,hem_specific_alignment=False,apply_polydata_transform=True)
#aligner.transform_folders(output_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = mg50_txmat,hem_specific_alignment=False,apply_polydata_transform=True)


In [ ]:
# # tranlate all data to produce grid 4x4
mg49_txmat = [0.994035, 0.109059, 0, 0, -0.109059, 0.994035, 0, 0, 0, 0, 1, 0, 453.964, -1331.68, 0, 1]
mg50_txmat = [0.996434, -0.0843786, 0, 0, 0.0843786, 0.996434, 0, 0, 0, 0, 0.999999, 0, -2113.88, -383.051, 0, 1]

row = 1
col = 0
# for row in range(0,2):
#     for col in range(0,2):

        #txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, col*col_offset,row*row_offset,0,1]

output_path = output_root+'Data_Grid/Row_{}_Col_{}/'.format(row+1,col+1)
#pathlib.Path(output_path).mkdir(exist_ok=True)

#         aligner = Alignment()
#         aligner.transform_folders(output_path,output_path,exp_name='MG49',icp=None,txmat = mg49_txmat,hem_specific_alignment=False,apply_polydata_transform=True)
#         aligner.transform_folders(output_path+'/Landmarks/', output_path +'/Landmarks/',exp_name='MG49',icp=None,txmat = mg49_txmat,hem_specific_alignment=False,apply_polydata_transform=True)
#         #aligner.transform_folders(output_path+'/Average/',output_path+'/Average/',exp_name='MG49',icp=None,txmat = mg49_txmat,hem_specific_alignment=False,apply_polydata_transform=True)
#         #aligner.transform_folders(output_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',exp_name='MG49',icp=None,txmat = mg49_txmat,hem_specific_alignment=False,apply_polydata_transform=True)

aligner = Alignment()
aligner.transform_folders(output_path,output_path,exp_name='MG50',icp=None,txmat = mg50_txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(output_path+'/Landmarks/', output_path +'/Landmarks/',exp_name='MG50',icp=None,txmat = mg50_txmat,hem_specific_alignment=False,apply_polydata_transform=True)
#aligner.transform_folders(output_path+'/Average/',output_path+'/Average/',exp_name='MG50',icp=None,txmat = mg50_txmat,hem_specific_alignment=False,apply_polydata_transform=True)
#aligner.transform_folders(output_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = mg50_txmat,hem_specific_alignment=False,apply_polydata_transform=True)


In [ ]:
# combine lhs rhs and find the right orientation for mg49 and mg 50
# lets do all these things now,
# combine surfaces
for row in range(2):
    for col in range(2):
        for exp in ['MG48','MG49','MG50']:
            for voxel_size in [50,100,200,300,400,450,500]:
                mg49 = Surface()
                lhs = Surface(output_root+'Data_Grid/Row_{}_Col_{}/Surfaces/{}_lhs_pia_open_bottom_voxel_size_{}.vtk'.format(row+1,col+1,exp,voxel_size))
                rhs = Surface(output_root+'Data_Grid/Row_{}_Col_{}/Surfaces/{}_rhs_pia_open_bottom_voxel_size_{}.vtk'.format(row+1,col+1,exp,voxel_size))
                mg49.append(lhs.surface)
                mg49.append(rhs.surface)
                mg49.write_surface_mesh(output_root+'Data_Grid/Row_{}_Col_{}/Surfaces/{}_pia_open_bottom_voxel_size_{}.vtk'.format(row+1,col+1,exp,voxel_size))
                
                mg49 = Surface()
                lhs = Surface(output_root+'Data_Grid/Row_{}_Col_{}/Surfaces/{}_lhs_pia_voxel_size_{}.vtk'.format(row+1,col+1,exp,voxel_size))
                rhs = Surface(output_root+'Data_Grid/Row_{}_Col_{}/Surfaces/{}_rhs_pia_voxel_size_{}.vtk'.format(row+1,col+1,exp,voxel_size))
                mg49.append(lhs.surface)
                mg49.append(rhs.surface)
                mg49.write_surface_mesh(output_root+'Data_Grid/Row_{}_Col_{}/Surfaces/{}_pia_voxel_size_{}.vtk'.format(row+1,col+1,exp,voxel_size))
            
            for br in ['vM1','vS1']:
                combined_landmarks = Landmarks()
                combined_landmarks.append_landmarks(Landmarks(output_root+'Data_Grid/Row_{}_Col_{}/Landmarks/K-Means/{}_lhs_rabies_kmeans_cluster_{}.landmarksAscii'.format(row+1,col+1,exp,br)).pts)
                combined_landmarks.append_landmarks(Landmarks(output_root+'Data_Grid/Row_{}_Col_{}/Landmarks/K-Means/{}_rhs_rabies_kmeans_cluster_{}.landmarksAscii'.format(row+1,col+1,exp,br)).pts)
                combined_landmarks.write_landmarks(output_root+'Data_Grid/Row_{}_Col_{}/Landmarks/K-Means/{}_rabies_kmeans_cluster_{}.landmarksAscii'.format(row+1,col+1,exp,br))
            
            

## Create Density cluster and images

In [ ]:
# Create density images
for br in ['vM1','vS1']:
    for row in [0,1]:
        for col in [0,1]:
            output_path = output_root+'Data_Grid/Row_{}_Col_{}/Landmarks/'.format(row+1,col+1)
            pathlib.Path(output_path+'Density_Clusters_New/').mkdir(exist_ok=True)
            for voxel_size in [50,100,200,300,400,450,500]:
                #for dens_cutoff_threshold in [0.15]:#[0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
                for exp_name in exp_names:
                    print(br,row,col,voxel_size,exp_name)
                    l = Landmarks(output_path+'K-Means/{}_rabies_kmeans_cluster_{}.landmarksAscii'.format(exp_name,br,))
                    
                    densimage = l.get_density_image(output_path+'Density_Clusters_New/{}_{}_rabies_landmarks_voxel_{}_cutoff_0.LandmarkAscii'.\
                                        format(exp_name,br,voxel_size),voxel_size=voxel_size)

In [ ]:
# create binary and contour images
for br in ['vM1','vS1']:
    for row in [0,1]:
        for col in [0,1]:
            output_path = output_root+'Data_Grid/Row_{}_Col_{}/Landmarks/Density_Clusters/'.format(row+1,col+1)
            #pathlib.Path(ouput_path).mkdir(exist_ok=True)
            for voxel_size in [400]:#[50,100,200,300,400,450,500]:
                for dens_cutoff_threshold in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
                    for exp_name in exp_names:
                        print(br,row,col,voxel_size,dens_cutoff_threshold,exp_name)
                        l = Landmarks(output_path+'{}_{}_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
                                      format(exp_name,br,voxel_size,dens_cutoff_threshold))
                        print(len(l.pts))
                        l.get_binary_and_contour_images_from_pts(output_file=output_path+'{}_{}_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
                                      format(exp_name,br,voxel_size,dens_cutoff_threshold),voxel_size=400,cutoff=dens_cutoff_threshold)
#             df_vM1_rabies_cluster = df_vM1_rabies_cluster.append(pd.DataFrame(columns=df_vM1_rabies_cluster.columns,\
#                                                                  data=[[exp_name,float(voxel_size),\
#                                                                         dens_cutoff_threshold*100,max_dens,threhold]]))

## Combine landmarks

In [ ]:
ouput_path = output_root+'Data_Grid/Row_{}_Col_{}/Landmarks/K-Means/'.format(2,2)
            #pathlib.Path(ouput_path).mkdir(exist_ok=True)
master_landmarks = Landmarks()
#for voxel_size in [400]:#[50,100,200,300,400,450,500]:
#    for dens_cutoff_threshold in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
for exp_name in exp_names:
    for br in ['vM1','vS1']:
        #for hem in ['lhs','rhs']:
        output_file=ouput_path+'{}_rabies_kmeans_cluster_{}.landmarksAscii'.\
                                                      format(exp_name,br)
        l = Landmarks(output_file)
        master_landmarks.append_landmarks(l.pts)
master_landmarks.write_landmarks(ouput_path+'All_Cortical_Landmarks.landmarksAscii')

In [ ]:
ouput_path = output_root+'Data_Grid/Row_{}_Col_{}/Landmarks/Density_Clusters/'.format(2,2)
            #pathlib.Path(ouput_path).mkdir(exist_ok=True)
master_landmarks = Landmarks()
for voxel_size in [400]:#[50,100,200,300,400,450,500]:
    for dens_cutoff_threshold in [0.1]:#[0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
        for exp_name in exp_names:
            for br in ['vM1','vS1']:
                print(exp_name,br)
                output_file=ouput_path+'{}_{}_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
                                                              format(exp_name,br,str(voxel_size),str(dens_cutoff_threshold))
                l = Landmarks(output_file)
                print(len(l.pts))
                master_landmarks.append_landmarks(l.pts)
                print(len(master_landmarks.pts))
master_landmarks.write_landmarks(ouput_path+'All_Selected_Landmarks.landmarksAscii')

## Trim PCAs

In [ ]:
for row in range(2):
    for col in range(2):
        for br in ['vM1','vS1']:
            input_path = output_root+'Data_Grid/Row_{}_Col_{}/{}_Ref_Surfaces/'.format(row+1,col+1,br)
            for exp in exp_names:
                print(os.path.exists(input_path + exp + '_{}_surf_res_300_voxel_size_400_cutoff_0.1.vtk'.format(br)))
                surf = Surface(input_path + exp + '_{}_surf_res_300_voxel_size_400_cutoff_0.1.vtk'.format(br))
                print(surf.surface.GetNumberOfCells())
                #print(glob.glob(input_path+'/'+exp+'_surf_res_50_voxel_size_400_cutoff_0.1_{}_PCA_*.am'.format(br)))
                for file in glob.glob(input_path+'/'+exp+'_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_*.am'):
                    print(file)
                    sg = AmiraSpatialGraph(file,generic_graph=True)
                    print(len(sg.graph_data.edge_list))
                    trimmed_edge_sg,bla,ble = Surface().get_axis_field_within_instersecting_surface\
                                            (surf.surface,sg.graph_data.edge_list,trim_edge=True,vec_extension_len=10000)
                    trimmed_edge_sg.write_spatial_graph(file[:-3]+'_trimmed_{}.am'.format(br))

## Saggital view

In [ ]:
MG49_lhs_txmat = [0.994151, 0.108, 0, 0, -0.108, 0.994151, 0, 0, 0, 0, 1, 0, 419.132, 161.89, 0, 1]
MG50_lhs_txmat = [0.987115, 0.160014, 0, 0, -0.160014, 0.987115, 0, 0, 0, 0, 1, 0, -42.3223, 200.422, 0, 1]
MG49_rhs_txmat = [1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1]
MG50_rhs_txmat = [0.961603, 0.274445, 0, 0, -0.274445, 0.961603, 0, 0, 0, 0, 1, 0, 3112.96, 1431.46, 0, 1]

output_path = output_root+'Data_Grid/Row_{}_Col_{}_Seggital/'.format(1,1)
pathlib.Path(output_path).mkdir(exist_ok=True)

aligner = Alignment(lhs=True)
aligner.transform_folders(output_path,output_path,exp_name='MG49_lhs',icp=None,txmat = MG49_lhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)
aligner.transform_folders(output_path+'/Landmarks/', output_path +'/Landmarks/',exp_name='MG49_lhs',icp=None,txmat = MG49_lhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)
aligner.transform_folders(output_path+'/Average/',output_path+'/Average/',exp_name='MG49_lhs',icp=None,txmat = MG49_lhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)
aligner.transform_folders(output_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = MG49_lhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)

aligner = Alignment(lhs=False)
aligner.transform_folders(output_path,output_path,exp_name='MG49_rhs',icp=None,txmat = MG49_rhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)
aligner.transform_folders(output_path+'/Landmarks/', output_path +'/Landmarks/',exp_name='MG49_rhs',icp=None,txmat = MG49_rhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)
aligner.transform_folders(output_path+'/Average/',output_path+'/Average/',exp_name='MG49_rhs',icp=None,txmat = MG49_rhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)
aligner.transform_folders(output_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = MG49_rhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)

aligner = Alignment(lhs=True)
aligner.transform_folders(output_path,output_path,exp_name='MG50_lhs',icp=None,txmat = MG50_lhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)
aligner.transform_folders(output_path+'/Landmarks/', output_path +'/Landmarks/',exp_name='MG50_lhs',icp=None,txmat = MG50_lhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)
aligner.transform_folders(output_path+'/Average/',output_path+'/Average/',exp_name='MG50_lhs',icp=None,txmat = MG50_lhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)
aligner.transform_folders(output_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = MG50_lhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)

aligner = Alignment(lhs=False)
aligner.transform_folders(output_path,output_path,exp_name='MG50_rhs',icp=None,txmat = MG50_rhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)
aligner.transform_folders(output_path+'/Landmarks/', output_path +'/Landmarks/',exp_name='MG50_rhs',icp=None,txmat = MG50_rhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)
aligner.transform_folders(output_path+'/Average/',output_path+'/Average/',exp_name='MG50_rhs',icp=None,txmat = MG50_rhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)
aligner.transform_folders(output_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = MG50_rhs_txmat,hem_specific_alignment=True,apply_polydata_transform=True)


In [ ]:
surf = Surface()
surf.append((Surface(output_root+'Final_Data/Surfaces/MG48_lhs_pia_open_bottom_voxel_size_50.vtk').surface))
surf.append((Surface(output_root+'Final_Data/Surfaces/MG48_rhs_pia_open_bottom_voxel_size_50.vtk').surface))
bb = surf.surface.GetBounds()
x_range = bb[1]-bb[0]
y_range = bb[3]-bb[2]
z_range = bb[5]-bb[4]

input_path = output_root+'Final_Data/'
output_path = output_root+'Data_Grid/'
pathlib.Path(output_path).mkdir(exist_ok=True)

row_offset = x_range + x_range*0.25
col_offset = y_range + y_range*0.5
 
row = 1
col = 1
for row in range(0,2):
    for col in range(0,2):
        if not (row==0 and col==0):
            #txmat = [1,0,0,0, 0,-1,0,0, 0,0,-1,0, col*col_offset,row*row_offset,0,1]
            #txmat = [1,0,0,0, 0,-1,0,0, 0,0,-1,0, 0,0,0,1]
            #txmat = [0.0, 0.0, -1, 0, -1, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0, col*col_offset,row*row_offset*1.3,0,1]
            txmat = [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, col*col_offset*1.1,row*row_offset*1.3,0,1]
            input_path = output_root+'Data_Grid/Row_{}_Col_{}_Seggital/'.format(1,1)
            output_path = output_root+'Data_Grid/Row_{}_Col_{}_Seggital/'.format(row+1,col+1)
            pathlib.Path(output_path).mkdir(exist_ok=True)

            aligner = Alignment(lhs=True)
            aligner.transform_folders(input_path,output_path,icp=None,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
            aligner.transform_folders(input_path+'/Landmarks/', output_path +'/Landmarks/',icp=None,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
            aligner.transform_folders(input_path+'/Average/',output_path+'/Average/',icp=None,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
            aligner.transform_folders(input_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)


            aligner = Alignment(lhs=False)
           # txmat = [0.0, 0.0, -1, 0, -1, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0, col*col_offset ,row*row_offset*1.2,0,1]
            txmat = [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, col*col_offset*1.1,row*row_offset*1.3,0,1]
            aligner.transform_folders(input_path,output_path,icp=None,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
            aligner.transform_folders(input_path+'/Landmarks/', output_path +'/Landmarks/',icp=None,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
            aligner.transform_folders(input_path+'/Average/',output_path+'/Average/',icp=None,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)
            aligner.transform_folders(input_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = txmat,hem_specific_alignment=True,apply_polydata_transform=True)

    

In [ ]:
input_path = output_root+'Final_Data/'
output_path = output_root+'Data_Grid/'
pathlib.Path(output_path).mkdir(exist_ok=True)

row_offset = x_range + x_range*0.25
col_offset = y_range + y_range*0.5
 
row = 1
col = 1

aligner = Alignment()
txmat = [0.0, 0.0, -1, 0, -1, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0, col*col_offset*1.1,row*row_offset*1.2,0,1]
output_path = output_root+'Data_Grid/Row_{}_Col_{}_Seggital/Atlas_lhs'.format(row+1,col+1)
pathlib.Path(output_path).mkdir(exist_ok=True)
aligner.transform_folder(input_path+'/Atlas/',output_path,icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)

aligner = Alignment()
output_path = output_root+'Data_Grid/Row_{}_Col_{}_Seggital/Atlas_rhs'.format(row+1,col+1)
pathlib.Path(output_path).mkdir(exist_ok=True)
txmat = [0.0, 0.0, -1, 0, -1, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0, col*col_offset*1.1,row*row_offset*1.2+12000,0,1]
aligner.transform_folder(input_path+'/Atlas/',output_path,icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)


In [ ]:
ouput_path = output_root+'Data_Grid/Row_{}_Col_{}_Seggital/Landmarks/K-Means/'.format(2,2)
            #pathlib.Path(ouput_path).mkdir(exist_ok=True)

#for voxel_size in [400]:#[50,100,200,300,400,450,500]:
#    for dens_cutoff_threshold in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
for hem in ['lhs','rhs']:
    master_landmarks = Landmarks()
    for exp_name in ['MG48','MG49','MG50']:
        for br in ['vM1','vS1']:

            output_file=ouput_path+'{}_{}_rabies_kmeans_cluster_{}.landmarksAscii'.\
                                                          format(exp_name,hem,br)
            l = Landmarks(output_file)
            master_landmarks.append_landmarks(l.pts)
    master_landmarks.write_landmarks(ouput_path+'All_Cortical_Landmarks_{}.landmarksAscii'.format(hem))

In [ ]:
ouput_path = output_root+'Data_Grid/Row_{}_Col_{}_Seggital/Landmarks/Density_Clusters/'.format(2,2)
            #pathlib.Path(ouput_path).mkdir(exist_ok=True)
for hem in ['lhs','rhs']:
    master_landmarks = Landmarks()
    for voxel_size in [400]:#[50,100,200,300,400,450,500]:
        for dens_cutoff_threshold in [0.1]:#[0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
            for exp_name in ['MG48','MG49','MG50']:
                for br in ['vM1','vS1']:
                    print(exp_name,br)
                    output_file=ouput_path+'{}_{}_{}_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
                                                                  format(exp_name,hem,br,str(voxel_size),str(dens_cutoff_threshold))
                    l = Landmarks(output_file)
                    print(len(l.pts))
                    master_landmarks.append_landmarks(l.pts)
                    print(len(master_landmarks.pts))
    master_landmarks.write_landmarks(ouput_path+'All_Selected_Landmarks_{}.landmarksAscii'.format(hem))

In [ ]:
# Create density images
for br in ['vM1','vS1']:
    for row in [0,1]:
        for col in [0,1]:
            output_path = output_root+'Data_Grid/Row_{}_Col_{}_Seggital/Landmarks/'.format(row+1,col+1)
            pathlib.Path(output_path+'Density_Clusters_New/').mkdir(exist_ok=True)
            for voxel_size in [50,100,200,300,400,450,500]:
                #for dens_cutoff_threshold in [0.15]:#[0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
                for exp_name in exp_names:
                    print(br,row,col,voxel_size,exp_name)
                    l = Landmarks(output_path+'K-Means/{}_rabies_kmeans_cluster_{}.landmarksAscii'.format(exp_name,br,))
                    
                    densimage = l.get_density_image(output_path+'Density_Clusters/{}_{}_rabies_landmarks_voxel_{}_cutoff_0.LandmarkAscii'.\
                                        format(exp_name,br,voxel_size),voxel_size=voxel_size)

In [ ]:
# create binary and contour images
for br in ['vM1','vS1']:
    for row in [0,1]:
        for col in [0,1]:
            output_path = output_root+'Data_Grid/Row_{}_Col_{}_Seggital/Landmarks/Density_Clusters/'.format(row+1,col+1)
            #pathlib.Path(ouput_path).mkdir(exist_ok=True)
            for voxel_size in [400]:#[50,100,200,300,400,450,500]:
                for dens_cutoff_threshold in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
                    for exp_name in exp_names:
                        print(br,row,col,voxel_size,dens_cutoff_threshold,exp_name)
                        l = Landmarks(output_path+'{}_{}_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
                                      format(exp_name,br,voxel_size,dens_cutoff_threshold))
                        print(len(l.pts))
                        l.get_binary_and_contour_images_from_pts(output_file=output_path+'{}_{}_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
                                      format(exp_name,br,voxel_size,dens_cutoff_threshold),voxel_size=400,cutoff=dens_cutoff_threshold)

In [ ]:
output_path = output_root+'Data_Grid/Row_{}_Col_{}_Seggital/Landmarks/Density_Clusters/'.format(0+1,1+1)
l = Landmarks(output_path+'{}_{}_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
                                      format('MG49_lhs','vM1',400,0.1))
l.get_binary_and_contour_images_from_pts(output_file=output_path+'{}_{}_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
                                      format('MG49_lhs','vM1',400,0.1),voxel_size=400,cutoff=0.1,holefillrad=1)

In [ ]:
for row in range(2):
    for col in range(2):
        for br in ['vM1','vS1']:
            input_path = output_root+'Data_Grid/Row_{}_Col_{}_Seggital/{}_Ref_Surfaces/'.format(row+1,col+1,br)
            for exp in exp_names:
                print(os.path.exists(input_path + exp + '_{}_surf_res_300_voxel_size_400_cutoff_0.1.vtk'.format(br)))
                surf = Surface(input_path + exp + '_{}_surf_res_300_voxel_size_400_cutoff_0.1.vtk'.format(br))
                print(surf.surface.GetNumberOfCells())
                #print(glob.glob(input_path+'/'+exp+'_surf_res_50_voxel_size_400_cutoff_0.1_{}_PCA_*.am'.format(br)))
                for file in glob.glob(input_path+'/'+exp+'_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_*.am'):
                    print(file)
                    sg = AmiraSpatialGraph(file,generic_graph=True)
                    print(len(sg.graph_data.edge_list))
                    trimmed_edge_sg,bla,ble = Surface().get_axis_field_within_instersecting_surface\
                                            (surf.surface,sg.graph_data.edge_list,trim_edge=True,vec_extension_len=10000)
                    trimmed_edge_sg.write_spatial_graph(file[:-3]+'_trimmed_{}.am'.format(br))

## Coronal View

In [ ]:
# Clip vM1 and vS1 for better coronal visualization
txmat = [1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, 1]

surf = Surface()
surf.append((Surface(output_root+'Final_Data/Surfaces/MG48_lhs_pia_open_bottom_voxel_size_50.vtk').surface))
surf.append((Surface(output_root+'Final_Data/Surfaces/MG48_rhs_pia_open_bottom_voxel_size_50.vtk').surface))
bb = surf.surface.GetBounds()
x_range = bb[1]-bb[0]
y_range = bb[3]-bb[2]
z_range = bb[5]-bb[4]

input_path = output_root+'Final_Data/'
output_path = output_root+'Data_Grid/'
pathlib.Path(output_path).mkdir(exist_ok=True)

row_offset = x_range + x_range*0.25
col_offset = y_range + y_range*0.25
 
row = 1
col = 1
for row in range(0,2):
    for col in range(0,2):
        #if not (row==0 and col==0):
        #txmat = [1,0,0,0, 0,-1,0,0, 0,0,-1,0, col*col_offset,row*row_offset,0,1]
        #txmat = [1,0,0,0, 0,-1,0,0, 0,0,-1,0, 0,0,0,1]
        #txmat = [0.0, 0.0, -1, 0, -1, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0, col*col_offset,row*row_offset*1.3,0,1]
        txmat = [1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, col*col_offset*1.3,row*row_offset*1.3,0,1]
        input_path = output_root+'Data_Grid/Row_{}_Col_{}/'.format(1,1)
        output_path = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_vM1/'.format(row+1,col+1)
        pathlib.Path(output_path).mkdir(exist_ok=True)
        #pathlib.Path(output_path+'/Surfaces/').mkdir(exist_ok=True)
        aligner = Alignment()
        #aligner.transform_folder(input_path+'/Surfaces/',output_path+'/Surfaces/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
        aligner.transform_folders(input_path,output_path,icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
        aligner.transform_folders(input_path+'/Landmarks/', output_path +'/Landmarks/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
        aligner.transform_folders(input_path+'/Average/',output_path+'/Average/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
        aligner.transform_folders(input_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)

        txmat = [1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, col*col_offset*1.3,row*row_offset*1.3+11000,0,1]
        input_path = output_root+'Data_Grid/Row_{}_Col_{}/'.format(1,1)
        output_path = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_vS1/'.format(row+1,col+1)
        pathlib.Path(output_path).mkdir(exist_ok=True)
        #pathlib.Path(output_path+'/Surfaces/').mkdir(exist_ok=True)
        aligner = Alignment()
        #aligner.transform_folder(input_path+'/Surfaces/',output_path+'/Surfaces/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
        aligner.transform_folders(input_path,output_path,icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
        aligner.transform_folders(input_path+'/Landmarks/', output_path +'/Landmarks/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
        aligner.transform_folders(input_path+'/Average/',output_path+'/Average/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
        aligner.transform_folders(input_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)


In [ ]:
def get_coronal_clipping_plane(surf_lhs,surf_rhs):
    ''' For the given surface find the y extent and 
    create clipping planes at the end points '''
    surf = Surface()
    surf.append(surf_lhs)
    surf.append(surf_rhs)
    
    bounds  = surf.surface.GetBounds()
    cutting_thickness = (bounds[4] - bounds[5])/2
    
    landmarks_center = [(bounds[0] + bounds[1])/2,(bounds[2] + bounds[3])/2,(bounds[4] + bounds[5])/2]
    print(landmarks_center,cutting_thickness)
    
    pt1 = [landmarks_center[0],landmarks_center[1],landmarks_center[2]+cutting_thickness]
    pt2 = [landmarks_center[0]+1000,landmarks_center[1],landmarks_center[2]+cutting_thickness]
    pt3 = [landmarks_center[0],landmarks_center[1]+1000,landmarks_center[2]+cutting_thickness]
    plane_front,tri_front = Surface().get_cutting_plane(pt1,pt2,pt3)
    #Surface(polydata=tri_front).write_surface_mesh(output_root+'bla.vtk')
    #Surface(polydata=pia1).write_surface_mesh(output_root+'b1.vtk')
    pt1 = [landmarks_center[0],landmarks_center[1],landmarks_center[2]-cutting_thickness]
    pt2 = [landmarks_center[0]+1000,landmarks_center[1],landmarks_center[2]-cutting_thickness]
    pt3 = [landmarks_center[0],landmarks_center[1]+1000,landmarks_center[2]-cutting_thickness]
    plane_back,tri_back = Surface().get_cutting_plane(pt1,pt2,pt3)
    
    return plane_front,plane_back,tri_front,tri_back

In [ ]:
def clip_surfaces(input_path,exp,clipping_plane_front,clipping_plane_back):
    
    pia_lhs = Surface(input_path+'/Surfaces/{}_lhs_pia_open_bottom_voxel_size_300.vtk'.format(exp))
    wm_lhs = Surface(input_path+'/Surfaces/{}_lhs_WM_open_bottom_voxel_size_300.vtk'.format(exp))
    pia_rhs = Surface(input_path+'/Surfaces/{}_rhs_pia_open_bottom_voxel_size_300.vtk'.format(exp))
    wm_rhs = Surface(input_path+'/Surfaces/{}_rhs_WM_open_bottom_voxel_size_300.vtk'.format(exp))

    pia1_lhs,wm1_lhs = Surface().clip_surfaces(pia_lhs.surface,wm_lhs.surface,clipping_plane_front,get_left=True)
    pia2_lhs,wm2_lhs = Surface().clip_surfaces(pia1_lhs,wm1_lhs,clipping_plane_back,get_left=False)
    Surface(polydata=pia2_lhs).write_surface_mesh(input_path+'/Surfaces/{}_lhs_pia_open_bottom_voxel_size_300_clipped.vtk'.format(exp))
    Surface(polydata=wm2_lhs).write_surface_mesh(input_path+'/Surfaces/{}_lhs_WM_open_bottom_voxel_size_300_clipped.vtk'.format(exp))

    pia1_rhs,wm1_rhs = Surface().clip_surfaces(pia_rhs.surface,wm_rhs.surface,clipping_plane_front,get_left=True)
    pia2_rhs,wm2_rhs = Surface().clip_surfaces(pia1_rhs,wm1_rhs,clipping_plane_back,get_left=False)
    Surface(polydata=pia2_rhs).write_surface_mesh(input_path+'/Surfaces/{}_rhs_pia_open_bottom_voxel_size_300_clipped.vtk'.format(exp))
    Surface(polydata=wm2_rhs).write_surface_mesh(input_path+'/Surfaces/{}_rhs_WM_open_bottom_voxel_size_300_clipped.vtk'.format(exp))

    return pia2_lhs,pia2_rhs

In [ ]:
# determine the cutting planes for coronal view.. need the center and the thickness
# thickness is that of the vS1 or vM1 surface
for row in range(0,2):
    for col in range(0,2): 
        
        if row==0 and col==0:
            exp = 'MG48'
        elif row==0 and col==1:
            exp = 'MG49'
        elif row==1 and col==0:
            exp = 'MG50'
            
        #input_path_vm1 = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_vM1/'.format(row+1,col+1)
        #input_path_vs1 = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_vS1/'.format(row+1,col+1)
        
        for br in ['vM1','vS1']:
            input_path = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_{}/'.format(row+1,col+1,br)
            if br == 'vM1':
                surf_vm1_lhs = Surface(input_path+'/{}_Ref_Surfaces/{}_lhs_{}_surf_res_300_voxel_size_400_cutoff_0.1.vtk'.format(br,exp,br))
                surf_vm1_rhs = Surface(input_path+'/{}_Ref_Surfaces/{}_rhs_{}_surf_res_300_voxel_size_400_cutoff_0.1.vtk'.format(br,exp,br))
                plane_front,plane_back,vm1_tri_front,vm1_tri_back = get_coronal_clipping_plane(surf_vm1_lhs.surface,surf_vm1_rhs.surface)
            else:
                surf_vs1_lhs = Surface(input_path+'/{}_Ref_Surfaces/{}_lhs_{}_surf_res_300_voxel_size_400_cutoff_0.1.vtk'.format(br,exp,br))
                surf_vs1_rhs = Surface(input_path+'/{}_Ref_Surfaces/{}_rhs_{}_surf_res_300_voxel_size_400_cutoff_0.1.vtk'.format(br,exp,br))
                plane_front,plane_back,vs1_tri_front,vs1_tri_back = get_coronal_clipping_plane(surf_vs1_lhs.surface,surf_vs1_rhs.surface)

            #Surface(polydata=vm1_tri_front).write_surface_mesh(output_root+'vm1_tri_front.vtk')
            #Surface(polydata=vm1_tri_back).write_surface_mesh(output_root+'vm1_tri_back.vtk')
            #Surface(polydata=vs1_tri_front).write_surface_mesh(output_root+'vs1_tri_front.vtk')
            #Surface(polydata=vs1_tri_back).write_surface_mesh(output_root+'vs1_tri_back.vtk')
            
            pia2_lhs,pia2_rhs = clip_surfaces(input_path,exp,plane_front,plane_back)
            #print(pia2_rhs.GetNumberOfCells())
            # clip rabies landmarks
            cortical_rabies_lhs = Landmarks(input_path+'/Landmarks/K-Means/{}_lhs_rabies_kmeans_cluster_{}.landmarksAscii'.format(exp,br))
            cortical_rabies_rhs = Landmarks(input_path+'/Landmarks/K-Means/{}_rhs_rabies_kmeans_cluster_{}.landmarksAscii'.format(exp,br))
            
            #print(pia2_lhs.GetBounds())
            #print(len(cortical_rabies_rhs.pts))
            clipped_landmarks_lhs = cortical_rabies_lhs.get_landmarks_in_bounding_box(pia2_lhs.GetBounds())
            clipped_landmarks_rhs = cortical_rabies_rhs.get_landmarks_in_bounding_box(pia2_rhs.GetBounds())
            Landmarks(pts = clipped_landmarks_lhs).write_landmarks(input_path+'/Landmarks/{}_lhs_rabies_cortical_landmarks_clipped.LandmarkAscii'.format(exp))
            Landmarks(pts = clipped_landmarks_rhs).write_landmarks(input_path+'/Landmarks/{}_rhs_rabies_cortical_landmarks_clipped.LandmarkAscii'.format(exp))
            #print(len(clipped_landmarks_lhs))
        
    

In [ ]:
# clip avg surface
row=1 
col=1

for br in ['vM1','vS1']:
    
    input_path = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_{}/'.format(row+1,col+1,br)
        
    surf_vm1_lhs = Surface(input_path+'/Average/{}/Avg_{}_lhs.vtk'.format(br,br))
    surf_vm1_rhs = Surface(input_path+'/Average/{}/Avg_{}_rhs.vtk'.format(br,br))
    plane_front,plane_back,vm1_tri_front,vm1_tri_back = get_coronal_clipping_plane(surf_vm1_lhs.surface,surf_vm1_rhs.surface)

    pia_lhs = Surface(input_path+'/Average/Avg_Pia_lhs_delunay.vtk')
    wm_lhs = Surface(input_path+'/Average/Avg_WM_lhs_delunay.vtk')
    pia_rhs = Surface(input_path+'/Average/Avg_Pia_rhs_delunay.vtk')
    wm_rhs = Surface(input_path+'/Average/Avg_WM_rhs_delunay.vtk')

    pia1_lhs,wm1_lhs = Surface().clip_surfaces(pia_lhs.surface,wm_lhs.surface,plane_front,get_left=True)
    pia2_lhs,wm2_lhs = Surface().clip_surfaces(pia1_lhs,wm1_lhs,plane_back,get_left=False)
    Surface(polydata=pia2_lhs).write_surface_mesh(input_path+'/Average/Avg_Pia_lhs_delunay_clipped.vtk')
    Surface(polydata=wm2_lhs).write_surface_mesh(input_path+'/Average/Avg_WM_lhs_delunay_clipped.vtk')

    pia1_rhs,wm1_rhs = Surface().clip_surfaces(pia_rhs.surface,wm_rhs.surface,plane_front,get_left=True)
    pia2_rhs,wm2_rhs = Surface().clip_surfaces(pia1_rhs,wm1_rhs,plane_back,get_left=False)
    Surface(polydata=pia2_rhs).write_surface_mesh(input_path+'/Average/Avg_Pia_rhs_delunay_clipped.vtk')
    Surface(polydata=wm2_rhs).write_surface_mesh(input_path+'/Average/Avg_WM_rhs_delunay_clipped.vtk')
    
    
    

In [ ]:
# clip the atlas surfaces
row=1 
col=1

for br in ['vM1','vS1']:
    
    input_path = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_{}/'.format(row+1,col+1,br)
    
    selected_landmarks = Landmarks()
    
    # read combined landmarks how else are we
    landmarks_lhs = Landmarks()
    landmarks_lhs.append_landmarks(Landmarks(input_path+'/Landmarks/Density_Clusters/MG48_lhs_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(br)).pts)
    landmarks_lhs.append_landmarks(Landmarks(input_path+'/Landmarks/Density_Clusters/MG49_lhs_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(br)).pts)
    landmarks_lhs.append_landmarks(Landmarks(input_path+'/Landmarks/Density_Clusters/MG50_lhs_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(br)).pts)
    selected_landmarks.append_landmarks(landmarks_lhs.pts)
    
    landmarks_rhs = Landmarks()
    landmarks_rhs.append_landmarks(Landmarks(input_path+'/Landmarks/Density_Clusters/MG48_rhs_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(br)).pts)
    landmarks_rhs.append_landmarks(Landmarks(input_path+'/Landmarks/Density_Clusters/MG49_rhs_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(br)).pts)
    landmarks_rhs.append_landmarks(Landmarks(input_path+'/Landmarks/Density_Clusters/MG50_rhs_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(br)).pts)
    selected_landmarks.append_landmarks(landmarks_rhs.pts)
    
    selected_landmarks.write_landmarks(input_path+'/Landmarks/Selected_Landmarks_{}.landmarksAscii'.format(br))
    
    surf_vm1_lhs = Surface(pts=landmarks_lhs.pts)
    surf_vm1_rhs = Surface(pts=landmarks_rhs.pts)
    plane_front,plane_back,vm1_tri_front,vm1_tri_back = get_coronal_clipping_plane(surf_vm1_lhs.surface,surf_vm1_rhs.surface)

    for file in glob.glob(input_path+'/Atlas/*.vtk'):
        if file.find('_clipped.vtk')<0:
            surf = Surface(file)
            pia1_lhs,wm1_lhs = Surface().clip_surfaces(surf.surface,surf.surface,plane_front,get_left=True)
            pia2_lhs,wm2_lhs = Surface().clip_surfaces(pia1_lhs,wm1_lhs,plane_back,get_left=False)
            Surface(polydata=pia2_lhs).write_surface_mesh(file+'_clipped.vtk')
        #Surface(polydata=wm2_lhs).write_surface_mesh(input_path+'/Average/Avg_WM_lhs_delunay_clipped.vtk')
    
    bounding_surf = Surface()
    atlas_pia = Surface(input_path+'/Atlas/atlas_pia.vtk')
    pia1,wm1 = Surface().clip_surfaces(atlas_pia.surface,atlas_pia.surface,plane_front,get_left=True)
    pia2,wm2 = Surface().clip_surfaces(pia1,wm1,plane_back,get_left=False)
    
    all_landmarks = Landmarks()
    for exp in ['MG48','MG49','MG50',]:
        l  = Landmarks(input_path+'/Landmarks/K-Means/{}_lhs_rabies_kmeans_cluster_{}.landmarksAscii'.format(exp,br))
        clipped_landmarks_lhs = l.get_landmarks_in_bounding_box(pia2.GetBounds())
        Landmarks(pts=clipped_landmarks_lhs).write_landmarks(input_path+'/Landmarks/{}_lhs_rabies_kmeans_cluster_{}_clipped.landmarksAscii'.format(exp,br))
        all_landmarks.append_landmarks(clipped_landmarks_lhs)
    for exp in ['MG48','MG49','MG50',]:
        l  = Landmarks(input_path+'/Landmarks/K-Means/{}_rhs_rabies_kmeans_cluster_{}.landmarksAscii'.format(exp,br))
        clipped_landmarks_rhs = l.get_landmarks_in_bounding_box(surf_vm1_rhs.surface.GetBounds())
        Landmarks(pts=clipped_landmarks_rhs).write_landmarks(input_path+'/Landmarks/{}_rhs_rabies_kmeans_cluster_{}_clipped.landmarksAscii'.format(exp,br))
        all_landmarks.append_landmarks(clipped_landmarks_rhs)
        
    all_landmarks.write_landmarks(input_path+'/Landmarks/All_Landmarks_{}.landmarksAscii'.format(br))

In [ ]:
# combine surfaces and landmarks
for row in range(0,2):
    for col in range(0,2): 
        
        if row==0 and col==0:
            exp = 'MG48'
        elif row==0 and col==1:
            exp = 'MG49'
        elif row==1 and col==0:
            exp = 'MG50'
            
        #input_path_vm1 = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_vM1/'.format(row+1,col+1)
        #input_path_vs1 = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_vS1/'.format(row+1,col+1)
        for br in ['vM1','vS1']:
            input_path = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_{}/'.format(row+1,col+1,br)
            surf = Surface()
            surf.append(Surface(input_path+'/Surfaces/{}_lhs_pia_open_bottom_voxel_size_300_clipped.vtk'.format(exp)).surface)
            surf.append(Surface(input_path+'/Surfaces/{}_rhs_pia_open_bottom_voxel_size_300_clipped.vtk'.format(exp)).surface)
            surf.write_surface_mesh(input_path+'/Surfaces/{}_pia_open_bottom_voxel_size_300_clipped.vtk'.format(exp))
            
            l = Landmarks()
            l.append_landmarks(Landmarks(input_path+'/Landmarks/{}_lhs_rabies_cortical_landmarks_clipped.LandmarkAscii'.format(exp)).pts)
            l.append_landmarks(Landmarks(input_path+'/Landmarks/{}_rhs_rabies_cortical_landmarks_clipped.LandmarkAscii'.format(exp)).pts)
            l.write_landmarks(input_path+'/Landmarks/{}_rabies_cortical_landmarks_clipped.LandmarkAscii'.format(exp))

In [ ]:
# refine coronal slices for neater visualization
MG49_vs1_txmat = [0.987134, 0.159893, 0, 0, -0.159893, 0.987134, 0, 0, 0, 0, 1, 0, 2270.27, -2108.75, 0, 1]
MG49_vm1_txmat = [0.993647, 0.112536, 0, 0, -0.112536, 0.993647, 0, 0, 0, 0, 1, 0, 256.428, -1671.4, 0, 1]

txmat = MG49_vm1_txmat
input_path = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_vM1/'.format(1,2)
output_path = input_path
pathlib.Path(output_path).mkdir(exist_ok=True)
#pathlib.Path(output_path+'/Surfaces/').mkdir(exist_ok=True)
aligner = Alignment()
#aligner.transform_folder(input_path+'/Surfaces/',output_path+'/Surfaces/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path,output_path,icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path+'/Landmarks/', output_path +'/Landmarks/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path+'/Average/',output_path+'/Average/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)

txmat = MG49_vs1_txmat
input_path = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_vS1/'.format(1,2)
output_path = input_path
pathlib.Path(output_path).mkdir(exist_ok=True)
#pathlib.Path(output_path+'/Surfaces/').mkdir(exist_ok=True)
aligner = Alignment()
#aligner.transform_folder(input_path+'/Surfaces/',output_path+'/Surfaces/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path,output_path,icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path+'/Landmarks/', output_path +'/Landmarks/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path+'/Average/',output_path+'/Average/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)


In [ ]:
MG50_vs1_txmat = [0.995094, -0.0989328, 0, 0, 0.0989328, 0.995094, 0, 0, 0, 0, 1, 0, -3646, -371.609, 0.00244141, 1]
MG50_vm1_txmat = [0.99909, -0.0426411, 0, 0, 0.0426411, 0.99909, 0, 0, 0, 0, 1, 0, -1077.53, -206.254, 0, 1]

txmat = MG50_vm1_txmat
input_path = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_vM1/'.format(2,1)
output_path = input_path
pathlib.Path(output_path).mkdir(exist_ok=True)
#pathlib.Path(output_path+'/Surfaces/').mkdir(exist_ok=True)
aligner = Alignment()
#aligner.transform_folder(input_path+'/Surfaces/',output_path+'/Surfaces/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path,output_path,icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path+'/Landmarks/', output_path +'/Landmarks/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path+'/Average/',output_path+'/Average/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)

txmat = MG50_vs1_txmat
input_path = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_vS1/'.format(2,1)
output_path = input_path
pathlib.Path(output_path).mkdir(exist_ok=True)
#pathlib.Path(output_path+'/Surfaces/').mkdir(exist_ok=True)
aligner = Alignment()
#aligner.transform_folder(input_path+'/Surfaces/',output_path+'/Surfaces/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path,output_path,icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path+'/Landmarks/', output_path +'/Landmarks/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path+'/Average/',output_path+'/Average/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)
aligner.transform_folders(input_path+'/Average_equal_extents/',output_path+'/Average_equal_extents/',icp=None,txmat = txmat,hem_specific_alignment=False,apply_polydata_transform=True)


In [ ]:
# density clusters
# Create density images
for br in ['vM1','vS1']:
    for row in [0,1]:
        for col in [0,1]:
            output_path = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_{}/Landmarks/'.format(row+1,col+1,br)
            #pathlib.Path(output_path+'Density_Clusters_New/').mkdir(exist_ok=True)
            for voxel_size in [50,100,200,300,400,450,500]:
                #for dens_cutoff_threshold in [0.15]:#[0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
                for exp_name in exp_names:
                    print(br,row,col,voxel_size,exp_name)
                    l = Landmarks(output_path+'K-Means/{}_rabies_kmeans_cluster_{}.landmarksAscii'.format(exp_name,br,))
                    
                    densimage = l.get_density_image(output_path+'Density_Clusters/{}_{}_rabies_landmarks_voxel_{}_cutoff_0.LandmarkAscii'.\
                                        format(exp_name,br,voxel_size),voxel_size=voxel_size)

In [ ]:
# create binary and contour images
for br in ['vM1','vS1']:
    for row in [0,1]:
        for col in [0,1]:
            output_path = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_{}/Landmarks/Density_Clusters/'.format(row+1,col+1,br)
            #pathlib.Path(ouput_path).mkdir(exist_ok=True)
            for voxel_size in [400]:#[50,100,200,300,400,450,500]:
                for dens_cutoff_threshold in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
                    for exp_name in exp_names:
                        print(br,row,col,voxel_size,dens_cutoff_threshold,exp_name)
                        l = Landmarks(output_path+'{}_{}_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
                                      format(exp_name,br,voxel_size,dens_cutoff_threshold))
                        print(len(l.pts))
                        l.get_binary_and_contour_images_from_pts(output_file=output_path+'{}_{}_rabies_landmarks_voxel_{}_cutoff_{}.LandmarkAscii'.\
                                      format(exp_name,br,voxel_size,dens_cutoff_threshold),voxel_size=400,cutoff=dens_cutoff_threshold)

In [ ]:
# trim pca
for row in range(2):
    for col in range(2):
        for br in ['vM1','vS1']:
            input_path = output_root+'Data_Grid/Row_{}_Col_{}_Coronal_{}/{}_Ref_Surfaces/'.format(row+1,col+1,br,br)
            for exp in exp_names:
                print(os.path.exists(input_path + exp + '_{}_surf_res_300_voxel_size_400_cutoff_0.1.vtk'.format(br)))
                surf = Surface(input_path + exp + '_{}_surf_res_300_voxel_size_400_cutoff_0.1.vtk'.format(br))
                print(surf.surface.GetNumberOfCells())
                #print(glob.glob(input_path+'/'+exp+'_surf_res_50_voxel_size_400_cutoff_0.1_{}_PCA_*.am'.format(br)))
                for file in glob.glob(input_path+'/'+exp+'_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_*.am'):
                    if file.find('trimmed')<0:
                        print(file)
                        sg = AmiraSpatialGraph(file,generic_graph=True)
                        print(len(sg.graph_data.edge_list))
                        trimmed_edge_sg,bla,ble = Surface().get_axis_field_within_instersecting_surface\
                                                (surf.surface,sg.graph_data.edge_list,trim_edge=True,vec_extension_len=10000)
                        trimmed_edge_sg.write_spatial_graph(file[:-3]+'_trimmed_{}.am'.format(br))

# Quantification of Rabies Spread

## Rabies in Atlas  

In [ ]:
# X axis is BRs in the atlas
# y axis is number and percentage of neurons in each BR, before and after clustering
df_main = pd.DataFrame(columns=['Atlas_Region','Exp','Hem','Count'])
rabies_path = output_root+'/Final_Data/Landmarks/K-Means/'
rabies_path = output_root+'/Final_Data/Landmarks/Density_Clusters/'
atlas_path = output_root+'/Fig_1/Rabies_Distribution/Atlas/'
atlas_path = output_root+'/Fig_1/Rabies_Distribution/Atlas_Clustered/'

for exp in ['MG48','MG49','MG50']:
    for hem in ['lhs','rhs']:
        #for br in ['vM1','vS1']:
        #landmarks_vm1 = Landmarks(rabies_path+'{}_{}_rabies_kmeans_cluster_{}.landmarksAscii'.format(exp,hem,'vM1'))
        #landmarks_vs1 = Landmarks(rabies_path+'{}_{}_rabies_kmeans_cluster_{}.landmarksAscii'.format(exp,hem,'vS1'))
        landmarks_vm1 = Landmarks(rabies_path+'{}_{}_vM1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(exp,hem))
        landmarks_vs1 = Landmarks(rabies_path+'{}_{}_vS1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(exp,hem))
        landmarks = Landmarks()
        landmarks.append_landmarks(landmarks_vm1.pts)
        landmarks.append_landmarks(landmarks_vs1.pts)
        total_landmarks = 0
        print(len(landmarks.pts))
        print(exp,hem)
        for file in glob.glob(output_root+'/Final_Data/Atlas/'+'/*.vtk'):
            surf = Surface(file)
            landmarks_within = landmarks.get_landmarks_within_given_surface_using_vtk(surf.surface) 
            atlas_region = None
            print(os.path.basename(file))
            
            if os.path.basename(file).find('Pia.vtk')>=0 :
                print(file)
                #print(len(landmarks_within))
                atlas_region = 'Pia'
                landmarks.write_landmarks(atlas_path+'/Pia_{}_{}.landmarksAscii'.format(hem,exp))
                
                #total_landmarks=len(landmarks_within)
                
            elif os.path.basename(file).find('Insular')>=0 and os.path.basename(file).endswith(hem+'.vtk'):
                atlas_region = 'Insular'
                print(file)
                total_landmarks=total_landmarks+len(landmarks_within)
                landmarks.write_landmarks(atlas_path+'/Insular_{}_{}.landmarksAscii'.format(hem,exp))
                
                
            elif os.path.basename(file).find('M1')>=0 and os.path.basename(file).find(hem)>0:
                atlas_region = 'M1'
                total_landmarks=total_landmarks+len(landmarks_within)
                landmarks.write_landmarks(atlas_path+'/M1_{}_{}.landmarksAscii'.format(hem,exp))
                
                
            elif os.path.basename(file).find('M2')>=0 and os.path.basename(file).find(hem)>0:
                atlas_region = 'M2'
                total_landmarks=total_landmarks+len(landmarks_within)
                landmarks.write_landmarks(atlas_path+'/M2_{}_{}.landmarksAscii'.format(hem,exp))
                
                
            elif os.path.basename(file).find('S1_fl')>=0 and os.path.basename(file).find(hem)>0:
                atlas_region = 'S1_fl'
                total_landmarks=total_landmarks+len(landmarks_within)
                landmarks.write_landmarks(atlas_path+'/S1_fl_{}_{}.landmarksAscii'.format(hem,exp))
                
                
            elif os.path.basename(file).find('S1ULp')>=0 and os.path.basename(file).find(hem)>0:
                atlas_region = 'S1_ULp'
                total_landmarks=total_landmarks+len(landmarks_within)
                landmarks.write_landmarks(atlas_path+'/S1ULp_{}_{}.landmarksAscii'.format(hem,exp))
               
                
            elif os.path.basename(file).find('SBF')>=0 and os.path.basename(file).find(hem)>0:
                atlas_region = 'SBF'
                total_landmarks=total_landmarks+len(landmarks_within)
                landmarks.write_landmarks(atlas_path+'/SBF_{}_{}.landmarksAscii'.format(hem,exp))
                
                
            elif os.path.basename(file).find('S2')>=0 and os.path.basename(file).find(hem)>0:
                atlas_region = 'S2'
                total_landmarks=total_landmarks+len(landmarks_within)
                landmarks.write_landmarks(atlas_path+'/S2_{}_{}.landmarksAscii'.format(hem,exp))
                
                
            if atlas_region is not None:
                df = pd.DataFrame(columns=['Atlas_Region','Exp','Hem','Count'])
                df['Atlas_Region'] = [atlas_region]
                df['Count'] = [len(landmarks_within)]
                df['Exp'] = [exp]
                df['Hem'] = [hem]
                #df['BR'] = [br]
                df_main = df_main.append(df)
        
        others = total_landmarks-total_landmarks
        df = pd.DataFrame(columns=['Atlas_Region','Exp','Hem','Count'])
        df['Atlas_Region'] = ['Others']
        df['Count'] = [others]
        df['Exp'] = [exp]
        df['Hem'] = [hem]
        #df['BR'] = [br]
        df_main = df_main.append(df)
                
#df_main.to_csv(output_root+'Fig_1/Rabies_Distribution/Atlas/Atlas_Counts.csv')

In [ ]:
df_main = pd.DataFrame()
df_main_per = pd.DataFrame()
df_main['Atlas_Region'] = ['M1','M2','S1_ULp','SBF','S2','S1_fl','Insular','Others']
df_main_per['Atlas_Region'] = ['M1','M2','S1_ULp','SBF','S2','S1_fl','Insular','Others']

for hem in ['lhs','rhs']:
    for exp in ['MG48','MG49','MG50']:
        m1= 0
        m2= 0
        insular= 0
        s1_fl= 0
        s1_ulp= 0
        sbf= 0
        s2= 0
        others = 0
        pia_count = 0
        other_total = 0
        m1_per= 0
        m2_per= 0
        insular_per= 0
        s1_fl_per= 0
        s1_ulp_per= 0
        sbf_per= 0
        s2_per= 0
        others_per = 0
        pia_count = 0
        other_total = 0
        
        pia_count = len(Landmarks(output_root+'Fig_1/Rabies_Distribution/Atlas/'+'Pia_{}_{}.landmarksAscii.landmarkAscii'.\
                              format(hem,exp)).pts)
        for file in sorted(glob.glob(output_root+'Fig_1/Rabies_Distribution/Atlas/'+'*_{}_{}.landmarksAscii.landmarkAscii'.format(hem,exp))):
            l = Landmarks(file)
            region = os.path.basename(file).split('_')[0]
            print(region) 
            print(len(l.pts))
            if region != 'Pia':
                other_total = other_total + len(l.pts)
                
                if region == 'M1':
                    m1 = len(l.pts)
                    m1_per = 100*len(l.pts)/pia_count
                elif region == 'M2':
                    m2 = len(l.pts)
                    m2_per = 100*len(l.pts)/pia_count
                elif region == 'Insular':
                    insular = len(l.pts)
                    insular_per = 100*len(l.pts)/pia_count
                elif region == 'S1fl':
                    s1_fl = len(l.pts)
                    s1_fl_per = 100*len(l.pts)/pia_count
                elif region == 'S1ULp':
                    s1_ulp = len(l.pts)
                    s1_ulp_per = 100*len(l.pts)/pia_count
                elif region == 'SBF':
                    sbf = len(l.pts)
                    sbf_per = 100*len(l.pts)/pia_count
                elif region == 'S2':
                    s2 = len(l.pts)
                    s2_per = 100*len(l.pts)/pia_count
                
        print(pia_count)
        others = pia_count - other_total
        others_per = 100*others/pia_count
        
        df_main['{}_{}'.format(hem,exp)] = [m1,m2,s1_ulp,sbf,s2,s1_fl,insular,others]
        df_main_per['{}_{}'.format(hem,exp)] = [m1_per,m2_per,s1_ulp_per,sbf_per,s2_per,s1_fl_per,insular_per,others_per]
        
df_main.to_csv(output_root+'Fig_1/Rabies_Distribution/Atlas/Atlas_Counts.csv',index=False)
df_main_per.to_csv(output_root+'Fig_1/Rabies_Distribution/Atlas/Atlas_Counts_per.csv',index=False)

## Clustering

In [ ]:
df_landmarks = pd.DataFrame()
df_surf_areas = pd.DataFrame()
df_surf_vols = pd.DataFrame()

df_landmarks['Cutoff'] = ['100','90','80','70','60','50','40','30','20','10','0']
df_surf_areas['Cutoff'] = ['100','90','80','70','60','50','40','30','20','10','0']
df_surf_vols['Cutoff'] = ['100','90','80','70','60','50','40','30','20','10','0']
br = 'vM1'
for br in ['vM1','vS1']:
    for exp in ['MG48','MG49','MG50']:
        for hem in ['lhs','rhs']:
            landmarks_total = Landmarks(output_root + 'Final_Data/Landmarks/K-Means/{}_{}_rabies_kmeans_cluster_{}.landmarksAscii'.format(exp,hem,br))
            surf_total = Surface(output_root + 'Final_Data/Landmarks/Density_Clusters/{}_{}_{}_rabies_landmarks_voxel_400_cutoff_{}.LandmarkAscii_surface_raw.vtk'.format(exp,hem,br,0))
            landmarks_counts = []
            surf_areas = []
            volumes = []
            landmarks_counts_per = []
            surf_areas_per = []
            volumes_per = []
            for cutoff in ['1','0.9','0.8','0.7','0.6','0.5','0.4','0.3','0.2','0.1','0']:
                if cutoff == '0':
                    landmarks = Landmarks(output_root + 'Final_Data/Landmarks/K-Means/{}_{}_rabies_kmeans_cluster_{}.landmarksAscii'.format(exp,hem,br))
                    surf = Surface(output_root + 'Final_Data/Landmarks/Density_Clusters/{}_{}_{}_rabies_landmarks_voxel_400_cutoff_{}.LandmarkAscii_surface_raw.vtk'.format(exp,hem,br,cutoff))
                else:
                    landmarks = Landmarks(output_root + 'Final_Data/Landmarks/Density_Clusters/{}_{}_{}_rabies_landmarks_voxel_400_cutoff_{}.LandmarkAscii'.format(exp,hem,br,cutoff) )
                    surf = Surface(output_root + 'Final_Data/Landmarks/Density_Clusters/{}_{}_{}_rabies_landmarks_voxel_400_cutoff_{}.LandmarkAscii_surface.vtk'.format(exp,hem,br,cutoff))
                
                landmarks_counts.append(len(landmarks.pts))
                landmarks_counts_per.append(100*len(landmarks.pts)/len(landmarks_total.pts))
                surf_areas.append(surf.get_surface_properties(prop='area'))
                surf_areas_per.append(100*surf.get_surface_properties(prop='area')/surf_total.get_surface_properties(prop='area'))
                volumes.append(surf.get_surface_properties(prop='volume'))
                volumes_per.append(100*surf.get_surface_properties(prop='volume')/surf_total.get_surface_properties(prop='volume'))
            df_landmarks['{}_{}_{}'.format(exp,hem,br)] = landmarks_counts
            df_surf_areas['{}_{}_{}'.format(exp,hem,br)] = surf_areas
            df_surf_vols['{}_{}_{}'.format(exp,hem,br)] = volumes
            df_landmarks['{}_{}_{}_per'.format(exp,hem,br)] = landmarks_counts_per
            df_surf_areas['{}_{}_{}_per'.format(exp,hem,br)] = surf_areas_per
            df_surf_vols['{}_{}_{}_per'.format(exp,hem,br)] = volumes_per
            
df_landmarks.to_csv(output_root + 'Fig_1/Rabies_Distribution/Clustering/landmarks_counts.csv',index=False)
df_surf_areas.to_csv(output_root + 'Fig_1/Rabies_Distribution/Clustering/surf_areas.csv',index=False)
df_surf_vols.to_csv(output_root + 'Fig_1/Rabies_Distribution/Clustering/surf_volumes.csv',index=False)


In [ ]:
df_surf_vols_amira = pd.read_csv(output_root + 'Fig_1/Rabies_Distribution/Clustering/surf_volumes_from_amira.csv',\
                                 sep=';',thousands='.',index_col=False)
df_surf_vols_amira_per = pd.DataFrame()
df_surf_vols_amira_per['Cutoff'] = ['100','90','80','70','60','50','40','30','20','10','0']
for i in range(len(df_surf_vols_amira.columns)):
    if i > 0:
        col = df_surf_vols_amira[df_surf_vols_amira.columns[i]]
        df_surf_vols_amira_per[df_surf_vols_amira.columns[i]+'_per'] = col/col[10]*100
df_surf_vols_amira_per.to_csv(output_root + 'Fig_1/Rabies_Distribution/Clustering/surf_volumes_from_amira_per.csv',index=False)

## Variability

In [ ]:
def get_edge_len(pca,neg_pca):
    #print((pca.graph_data.edge_list[0][1],neg_pca.graph_data.edge_list[0][1]))
    return distance.euclidean(pca.graph_data.edge_list[0][1],neg_pca.graph_data.edge_list[0][1])

In [ ]:
def get_dist_from_ref_pt(ref_pt,pt):
    return [abs(ref_pt[0]-pt[0]),(ref_pt[1]-pt[1]),abs(ref_pt[2]-pt[2])]

In [ ]:
def get_2d_axis(edge,axis='XY'):
    edge_2d = []
    if axis=='XY':
        # check if edge needs to be reversed
        if edge[1][1] < edge[0][1]:
            edge_2d = [[edge[0][0],edge[0][1],0],[edge[1][0],edge[1][1],0]]
        else:
            edge_2d = [[edge[1][0],edge[1][1],0],[edge[0][0],edge[0][1],0]]
            
    elif axis=='XZ':
        if edge[1][1] < edge[0][1]:
            edge_2d = [[edge[0][0],0,edge[0][2]],[edge[1][0],0,edge[1][2]]]
        else:
            edge_2d = [[edge[1][0],0,edge[1][2]],[edge[0][0],0,edge[0][2]]]
    else:
        edge_2d = [[0,edge[0][1],edge[0][2]],[0,edge[1][1],edge[1][2]]]
    #print(edge_2d)
    return edge_2d

In [ ]:
# of surface areas, volumes, location and extents.
cols = ['Exp','Hem','BR','rabies_surf_area','surf_area','rabies_surf_vol','surf_vol',\
        'pia_surf_area','wm_surf_area',\
        'pca0_extent','pca1_extent','pca2_extent',\
        'pca0_angle_X_axis','pca0_angle_Y_axis','pca0_angle_Z_axis',\
        'pca1_angle_X_axis','pca1_angle_Y_axis','pca1_angle_Z_axis',\
        'pca2_angle_X_axis','pca2_angle_Y_axis','pca2_angle_Z_axis',\
        'Dist_From_Bregma_X','Dist_From_Bregma_Y','Dist_From_Bregma_Z',\
       ]

df_main = pd.DataFrame(columns=cols)
for br in ['vM1','vS1']:
    for hem in ['lhs','rhs']: 
        for exp in ['MG48','MG49','MG50']:
            
            df = pd.DataFrame()
            df['Exp'] = [exp]
            df['Hem'] = [hem]
            df['BR'] = [br]
            
            rabies_surf = Surface(output_root + 'Final_Data/Landmarks/Density_Clusters/{}_{}_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_surface.vtk'.format(exp,hem,br) )
            surf = Surface(output_root + 'Final_Data/{}_Ref_Surfaces/{}_{}_{}_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(br,exp,hem,br))
            surf_pia = Surface(output_root + 'Final_Data/{}_Ref_Surfaces/{}_{}_{}_pia_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(br,exp,hem,br.lower()))
            surf_wm = Surface(output_root + 'Final_Data/{}_Ref_Surfaces/{}_{}_{}_wm_surf_res_50_voxel_size_400_cutoff_0.1.vtk'.format(br,exp,hem,br.lower()))
            bregma = Landmarks(output_root + 'Final_Data/Atlas/Bregma.landmarkAscii').pts[0]
            centroid = Landmarks(output_root + 'Final_Data/Landmarks/Density_Clusters/{}_{}_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii_Centroid.LandmarkAscii'.format(exp,hem,br)).pts[0]
            
            df['rabies_surf_area'] = [rabies_surf.get_surface_properties(prop='area')]
            df['surf_area'] = [surf.get_surface_properties(prop='area')]
            df['rabies_surf_vol'] = [rabies_surf.get_surface_properties(prop='volume')]
            df['surf_vol'] = [surf.get_surface_properties(prop='volume')]
            df['pia_surf_area'] = [surf_pia.get_surface_properties(prop='area')]
            df['wm_surf_area'] = [surf_wm.get_surface_properties(prop='area')]
            
            # for pcas get angles from the visualization data grid so that the number matches the figures
            if br=='vM1':
                ext = 'trimmed_vM1'
            else:
                ext = 'trimmed_vS1'
            if exp == 'MG48':
                pca_path_top_view = output_root + 'Data_Grid/Row_1_Col_1/'
                pca_path_side_view = output_root + 'Data_Grid/Row_1_Col_1_Seggital/'
                if br == 'vM1':
                    pca_path_coronal_view = output_root + 'Data_Grid/Row_1_Col_1_Coronal_vM1/'
                else:
                    pca_path_coronal_view = output_root + 'Data_Grid/Row_1_Col_1_Coronal_vS1/'
                pca_path_side_view = output_root + 'Data_Grid/Row_1_Col_1_Seggital/'
            elif exp == 'MG49':
                pca_path_top_view = output_root + 'Data_Grid/Row_1_Col_2/'
                pca_path_side_view = output_root + 'Data_Grid/Row_1_Col_2_Seggital/'
                if br == 'vM1':
                    pca_path_coronal_view = output_root + 'Data_Grid/Row_1_Col_2_Coronal_vM1/'
                else:
                    pca_path_coronal_view = output_root + 'Data_Grid/Row_1_Col_2_Coronal_vS1/'
            elif exp == 'MG50':
                pca_path_top_view = output_root + 'Data_Grid/Row_2_Col_1/'
                pca_path_side_view = output_root + 'Data_Grid/Row_2_Col_1_Seggital/'
                if br == 'vM1':
                    pca_path_coronal_view = output_root + 'Data_Grid/Row_2_Col_1_Coronal_vM1/'
                else:
                    pca_path_coronal_view = output_root + 'Data_Grid/Row_2_Col_1_Coronal_vS1/'
                
            if br=='vS1' and hem=='rhs' and exp=='MG49':
                pca1 = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca1_neg = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2 = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_neg = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0 = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0_neg = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
            
                pca1_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca1_neg_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_neg_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0_neg_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                
                pca1_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca1_neg_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_neg_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0_neg_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
            
            elif br=='vS1' and hem=='lhs' and exp=='MG49':
                pca1 = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca1_neg = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0 = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0_neg = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2 = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_neg = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)

                pca1_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca1_neg_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0_neg_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_neg_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)

                pca1_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca1_neg_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0_neg_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_neg_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)

            else:
                pca0 = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0_neg = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca1 = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca1_neg = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2 = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_neg = AmiraSpatialGraph(pca_path_top_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)

                pca0_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0_neg_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca1_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca1_neg_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_neg_side = AmiraSpatialGraph(pca_path_side_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                
                pca0_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca0_neg_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca1_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca1_neg_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                pca2_neg_coronal = AmiraSpatialGraph(pca_path_coronal_view + '{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_Neg_{}.am'.format(br,exp,hem,ext),generic_graph=True)
                
            df['pca0_extent'] = [get_edge_len(pca0,pca0_neg)]
            df['pca1_extent'] = [get_edge_len(pca1,pca1_neg)]
            df['pca2_extent'] = [get_edge_len(pca2,pca2_neg)]
            
            df['pca0_angle_X_axis'] = [Vectors().get_angle_between_vectors([[0,0,0],[1,0,0]],get_2d_axis(pca0.graph_data.edge_list[0],axis='XY'),ignore_opposite_direction=False,keep_negative_angles=False)]
            df['pca0_angle_Y_axis'] = [Vectors().get_angle_between_vectors([[0,0,0],[0,-1,0]],get_2d_axis(pca0.graph_data.edge_list[0],axis='XY'),ignore_opposite_direction=False,keep_negative_angles=False)]
            df['pca0_angle_Z_axis'] = [Vectors().get_angle_between_vectors([[0,0,0],[1,0,0]],get_2d_axis(pca0_side.graph_data.edge_list[0],axis='XY'),ignore_opposite_direction=False,keep_negative_angles=False)]
            
            df['pca1_angle_X_axis'] = [Vectors().get_angle_between_vectors([[0,0,0],[1,0,0]],get_2d_axis(pca1.graph_data.edge_list[0],axis='XY'),ignore_opposite_direction=False,keep_negative_angles=False)]
            df['pca1_angle_Y_axis'] = [Vectors().get_angle_between_vectors([[0,0,0],[0,-1,0]],get_2d_axis(pca1.graph_data.edge_list[0],axis='XY'),ignore_opposite_direction=False,keep_negative_angles=False)]
            if br == 'vM1':
                df['pca1_angle_Z_axis'] = [Vectors().get_angle_between_vectors([[0,0,0],[1,0,0]],get_2d_axis(pca1_coronal.graph_data.edge_list[0],axis='XY'),ignore_opposite_direction=False,keep_negative_angles=False)]
            else:
                df['pca1_angle_Z_axis'] = [Vectors().get_angle_between_vectors([[0,0,0],[1,0,0]],get_2d_axis(pca1_side.graph_data.edge_list[0],axis='XY'),ignore_opposite_direction=False,keep_negative_angles=False)]
            
            df['pca2_angle_X_axis'] = [Vectors().get_angle_between_vectors([[0,0,0],[1,0,0]],get_2d_axis(pca2.graph_data.edge_list[0],axis='XY'),ignore_opposite_direction=False,keep_negative_angles=False)]
            df['pca2_angle_Y_axis'] = [Vectors().get_angle_between_vectors([[0,0,0],[0,-1,0]],get_2d_axis(pca2.graph_data.edge_list[0],axis='XY'),ignore_opposite_direction=False,keep_negative_angles=False)]
            if br == 'vM1':
                df['pca2_angle_Z_axis'] = [Vectors().get_angle_between_vectors([[0,0,0],[1,0,0]],get_2d_axis(pca2_side.graph_data.edge_list[0],axis='XY'),ignore_opposite_direction=False,keep_negative_angles=False)]
            else:
                if exp=='MG49' and br=='vS1' and hem=='rhs':
                    df['pca2_angle_Z_axis'] = [Vectors().get_angle_between_vectors([[0,0,0],[1,0,0]],get_2d_axis(pca0_coronal.graph_data.edge_list[0],axis='XY'),ignore_opposite_direction=False,keep_negative_angles=False)]
                else:
                    df['pca2_angle_Z_axis'] = [Vectors().get_angle_between_vectors([[0,0,0],[1,0,0]],get_2d_axis(pca2_coronal.graph_data.edge_list[0],axis='XY'),ignore_opposite_direction=False,keep_negative_angles=False)]
            
            #print(bregma,centroid)
            #print(Vectors().get_angle_between_vectors(pca0.graph_data.edge_list[0],pca1.graph_data.edge_list[0]))
            #print(Vectors().get_angle_between_vectors(pca1.graph_data.edge_list[0],pca2.graph_data.edge_list[0]))
            #print(Vectors().get_angle_between_vectors(pca2_side.graph_data.edge_list[0],pca0_side.graph_data.edge_list[0]))
            
            dist = get_dist_from_ref_pt(bregma,centroid)
            df['Dist_From_Bregma_X'] = [dist[0]]
            df['Dist_From_Bregma_Y'] = [dist[1]]
            df['Dist_From_Bregma_Z'] = [dist[2]]
            
            df_main = df_main.append(df)
            
df_main.to_csv(output_root + 'Fig_1/Rabies_Distribution/Variability/variability.csv',index=False)


In [ ]:
df_main['pca2_angle_Z_axis']

In [ ]:
pca0 = AmiraSpatialGraph(output_root + 'Final_Data/Average/vS1/Avg_vS1_lhs_PCA0.am',generic_graph=True)
#pca0_neg = AmiraSpatialGraph(output_root + 'Final_Data/{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_Neg.am'.format(br,exp,hem),generic_graph=True)
pca1 = AmiraSpatialGraph(output_root + 'Final_Data/Average//vS1/Avg_vS1_lhs_PCA1.am',generic_graph=True)
#pca1_neg = AmiraSpatialGraph(output_root + 'Final_Data/{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_Neg.am'.format(br,exp,hem),generic_graph=True)
pca2 = AmiraSpatialGraph(output_root + 'Final_Data/Average//vS1/Avg_vS1_lhs_PCA2.am',generic_graph=True)
#pca2_neg = AmiraSpatialGraph(output_root + 'Final_Data/{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_Neg.am'.format(br,exp,hem),generic_graph=True)

print(Vectors().get_angle_between_vectors(pca0.graph_data.edge_list[0],pca1.graph_data.edge_list[0]))
print(Vectors().get_angle_between_vectors(pca1.graph_data.edge_list[0],pca2.graph_data.edge_list[0]))
print(Vectors().get_angle_between_vectors(pca2.graph_data.edge_list[0],pca0.graph_data.edge_list[0]))

In [ ]:
pca0 = AmiraSpatialGraph(output_root + 'Final_Data/{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2.am'.format('vS1','MG48','rhs'),generic_graph=True)
#pca1 = AmiraSpatialGraph(output_root + 'Final_Data/{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1.am'.format('vM1','MG48','lhs'),generic_graph=True)
axis_2d_0 = get_2d_axis(pca0.graph_data.edge_list[0],axis='XY') 
#axis_2d_1 = get_2d_axis(pca1.graph_data.edge_list[0],axis='XY') 
sg = AmiraSpatialGraph(generic_graph=True)
sg.graph_data.add_edge(axis_2d_0[0],axis_2d_0[1])
#sg.graph_data.add_edge(axis_2d_1[0],axis_2d_1[1])
sg.write_spatial_graph(output_root+'pca01.am')

Vectors().get_angle_between_vectors([[0,0,0],[1,0,0]],axis_2d_0)

# Create Avg Layer borders

In [ ]:
# what is diff between mean of individual exp means and global means
mg48_lhs = pd.read_csv(output_root+'Final_Data/Layer_Borders/MG48_lhs_vM1_Borders.csv',index_col=False)
mg49_lhs = pd.read_csv(output_root+'Final_Data/Layer_Borders/MG49_lhs_vM1_Borders.csv',index_col=False)
mg50_lhs = pd.read_csv(output_root+'Final_Data/Layer_Borders/MG50_lhs_vM1_Borders.csv',index_col=False)
mg48_rhs = pd.read_csv(output_root+'Final_Data/Layer_Borders/MG48_rhs_vM1_Borders.csv',index_col=False)
mg49_rhs = pd.read_csv(output_root+'Final_Data/Layer_Borders/MG49_rhs_vM1_Borders.csv',index_col=False)
mg50_rhs = pd.read_csv(output_root+'Final_Data/Layer_Borders/MG50_rhs_vM1_Borders.csv',index_col=False)

np.array([mg48_lhs.mean(),mg48_rhs.mean(),mg49_lhs.mean(),mg49_rhs.mean(),mg50_lhs.mean(),mg50_rhs.mean()]).mean(axis=0)

In [ ]:
df = pd.read_csv(output_root+'Final_Data/Layer_Borders/vM1_Borders.csv',index_col=False)
df.mean()

In [ ]:
# Figure out whether lhs and rhs layers are different
df = pd.read_csv(output_root+'Final_Data/Layer_Borders/vM1_Borders.csv',index_col=False)
#df[(df['Exp_Name']=='MG48_lhs') & (df['Exp_Name']=='MG49_lhs') & (df['Exp_Name']=='MG50_lhs')]

l1_lhs_mean_per = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L1_Border_Per'].mean(axis=0)
l1_lhs_std_per = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L1_Border_Per'].std(axis=0)

l23_lhs_mean_per = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L2_3_Border_Per'].mean(axis=0)
l23_lhs_std_per = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L2_3_Border_Per'].std(axis=0)

l5a_lhs_mean_per = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L5A_Border_Per'].mean(axis=0)
l5a_lhs_std_per = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L5A_Border_Per'].std(axis=0)

l5b_lhs_mean_per = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L5B_Border_Per'].mean(axis=0)
l5b_lhs_std_per = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L5B_Border_Per'].std(axis=0)

l1_lhs_mean = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L1_Border'].mean(axis=0)
l1_lhs_std = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L1_Border'].std(axis=0)

l23_lhs_mean = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L2_3_Border'].mean(axis=0)
l23_lhs_std = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L2_3_Border'].std(axis=0)

l5a_lhs_mean = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L5A_Border'].mean(axis=0)
l5a_lhs_std = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L5A_Border'].std(axis=0)

l5b_lhs_mean = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L5B_Border'].mean(axis=0)
l5b_lhs_std = df[(df['Exp_Name']=='MG49_lhs') | (df['Exp_Name']=='MG48_lhs') | (df['Exp_Name']=='MG50_lhs') ]['L5B_Border'].std(axis=0)


In [ ]:
l1_rhs_mean_per = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L1_Border_Per'].mean(axis=0)
l1_rhs_std_per = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L1_Border_Per'].std(axis=0)

l23_rhs_mean_per = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L2_3_Border_Per'].mean(axis=0)
l23_rhs_std_per = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L2_3_Border_Per'].std(axis=0)

l5a_rhs_mean_per = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L5A_Border_Per'].mean(axis=0)
l5a_rhs_std_per = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L5A_Border_Per'].std(axis=0)

l5b_rhs_mean_per = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L5B_Border_Per'].mean(axis=0)
l5b_rhs_std_per = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L5B_Border_Per'].std(axis=0)

l1_rhs_mean = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L1_Border'].mean(axis=0)
l1_rhs_std = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L1_Border'].std(axis=0)

l23_rhs_mean = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L2_3_Border'].mean(axis=0)
l23_rhs_std = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L2_3_Border'].std(axis=0)

l5a_rhs_mean = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L5A_Border'].mean(axis=0)
l5a_rhs_std = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L5A_Border'].std(axis=0)

l5b_rhs_mean = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L5B_Border'].mean(axis=0)
l5b_rhs_std = df[(df['Exp_Name']=='MG49_rhs') | (df['Exp_Name']=='MG48_rhs') | (df['Exp_Name']=='MG50_rhs') ]['L5B_Border'].std(axis=0)


In [ ]:
l1_mean = df['L1_Border_Per'].mean(axis=0)
l1_std = df['L1_Border_Per'].std(axis=0)

l23_mean = df['L2_3_Border_Per'].mean(axis=0)
l23_std = df['L2_3_Border_Per'].std(axis=0)

l5a_mean = df['L5A_Border_Per'].mean(axis=0)
l5a_std = df['L5A_Border_Per'].std(axis=0)

l5b_mean = df['L5B_Border_Per'].mean(axis=0)
l5b_std = df['L5B_Border_Per'].std(axis=0)


In [ ]:
l1_mean,l23_mean,l5a_mean,l5b_mean, l1_std,l23_std,l5a_std,l5b_std

In [ ]:
l1_lhs_mean,l23_lhs_mean,l5a_lhs_mean,l5b_lhs_mean, l1_lhs_std,l23_lhs_std,l5a_lhs_std,l5b_lhs_std

In [ ]:
l1_rhs_mean,l23_rhs_mean,l5a_rhs_mean,l5b_rhs_mean, l1_rhs_std,l23_rhs_std,l5a_rhs_std,l5b_rhs_std

In [ ]:
mg50_rhs.mean()

## Create avg axis field

In [ ]:
# create avg axis field for the avg pia wm surfs
combined_lhs = AmiraSpatialGraph(output_root+'/Final_Data/Average/combined_axis_field_lhs.am',generic_graph=True)
combined_rhs = AmiraSpatialGraph(output_root+'/Final_Data/Average/combined_axis_field_rhs.am',generic_graph=True)
pia_lhs = Surface(output_root+'/Final_Data/Average/Avg_Pia_lhs_delunay.vtk')
pia_rhs = Surface(output_root+'/Final_Data/Average/Avg_Pia_rhs_delunay.vtk')
wm_lhs = Surface(output_root+'/Final_Data/Average/Avg_WM_lhs_delunay.vtk')
wm_rhs = Surface(output_root+'/Final_Data/Average/Avg_WM_rhs_delunay.vtk')
vm1_lhs_surf = Surface(output_root+'/Final_Data/Average/vM1/Avg_vM1_lhs.vtk')
vm1_rhs_surf = Surface(output_root+'/Final_Data/Average/vM1/Avg_vM1_rhs.vtk')

In [ ]:
# generate uniform axis field

hem  = 'rhs'

voxel_size = 400
cutoff=0.1
surf_res = 50

pia = Surface(output_root+'/Final_Data/Average/vM1/Avg_vM1_{}_Pia.vtk'.format(hem))
wm = Surface(output_root+'/Final_Data/Average/vM1/Avg_vM1_{}_WM.vtk'.format(hem))

axis_field = AmiraSpatialGraph(output_root+'/Final_Data/Average/combined_axis_field_{}.am'.format(hem),generic_graph=True).graph_data.edge_list

horizontal_axis = [AmiraSpatialGraph(output_root+'/Final_Data/Average/vM1/Avg_vM1_{}_PCA0_Trimmed.am'.format(hem),\
                                     generic_graph=True).graph_data.edge_list[0][1],\
                   AmiraSpatialGraph(output_root+'/Final_Data/Average/vM1/Avg_vM1_{}_PCA_0_Neg_Trimmed.am'.format(hem),\
                                     generic_graph=True).graph_data.edge_list[0][1]]
vertical_axis = [AmiraSpatialGraph(output_root+'/Final_Data/Average/vM1/Avg_vM1_{}_PCA2_Trimmed.am'.format(hem),\
                                     generic_graph=True).graph_data.edge_list[0][1],\
                   AmiraSpatialGraph(output_root+'/Final_Data/Average/vM1/Avg_vM1_{}_PCA_2_Neg_Trimmed.am'.format(hem),\
                                     generic_graph=True).graph_data.edge_list[0][1]]
side_axis = [AmiraSpatialGraph(output_root+'/Final_Data/Average/vM1/Avg_vM1_{}_PCA1_Trimmed.am'.format(hem),\
                                     generic_graph=True).graph_data.edge_list[0][1],\
                   AmiraSpatialGraph(output_root+'/Final_Data/Average/vM1/Avg_vM1_{}_PCA_1_Neg_Trimmed.am'.format(hem),\
                                     generic_graph=True).graph_data.edge_list[0][1]]
center_pt = AmiraSpatialGraph(output_root+'/Final_Data/Average/vM1/Avg_vM1_{}_PCA0_Trimmed.am'.format(hem),\
                                     generic_graph=True).graph_data.edge_list[0][0]

# make the grid
grid_pts = []
step_size = 500
radius = 800
uniform_axis_field = AmiraSpatialGraph(generic_graph=True)

output_path_vM1_axis_field = output_root+'/Final_Data/Average/vM1/Uniform_Axis_Field_{}_New/'.format(hem)
pathlib.Path((output_path_vM1_axis_field)).mkdir(exist_ok=True)

#uniform_axis_field.graph_data.add_edge(vertical_axis[0],vertical_axis[1])
# project the central axis to wm 
side_uv_left = Vectors().get_unit_vec(center_pt,side_axis[0])
side_uv_right = Vectors().get_unit_vec(center_pt,side_axis[1])
start_ind = 0
step_direction = -1
# extend the horzontal axis
#     horizontal_axis_begin = Vectors().create_pt_along_vector_at_given_distance(10000,horizontal_axis[1],horizontal_axis[0])
#     horizontal_axis_end = Vectors().create_pt_along_vector_at_given_distance(10000,horizontal_axis[0],horizontal_axis[1])

for end_pt in [horizontal_axis[0],horizontal_axis[1]]:
    step_direction = -1*step_direction
    row_ind = step_direction
    for i in range(start_ind,20):
        uniform_axis_field_rowwise = AmiraSpatialGraph(generic_graph=True)
        uniform_axis_field_rowwise_left = AmiraSpatialGraph(generic_graph=True)
        uniform_axis_field_rowwise_right = AmiraSpatialGraph(generic_graph=True)
        pt = Vectors().create_pt_along_vector_at_given_distance(i*step_size,center_pt,end_pt)
        uv,axis = get_local_avg_axis(axis_field,[pt],pt,pia,wm,radius)
        #print(axis)
        if axis is not None:
            uniform_axis_field.graph_data.add_edge(axis[0],axis[1])
            uniform_axis_field_rowwise.graph_data.add_edge(axis[0],axis[1])
            write_edge_sg(axis,output_path_vM1_axis_field+'_vM1_axis_field_row_{}_{}_{}_{}_{}.am'.\
                          format(row_ind,0,surf_res,voxel_size,cutoff))
            uniform_axis_field_rowwise_left.graph_data.add_edge(axis[0],axis[1])
            uniform_axis_field_rowwise_right.graph_data.add_edge(axis[0],axis[1])
        # now move along the minor axis

        for j in range(1,10):
            pt1 = (j*step_size * side_uv_left) + pt
            uv,axis = get_local_avg_axis(axis_field,[pt1],pt1,pia,wm,radius)
            if axis is not None:
                uniform_axis_field.graph_data.add_edge(axis[0],axis[1])
                uniform_axis_field_rowwise.graph_data.add_edge(axis[0],axis[1])
                write_edge_sg(axis,output_path_vM1_axis_field+'_vM1_axis_field_row_{}_{}_{}_{}_{}.am'.\
                              format(row_ind,-j,surf_res,voxel_size,cutoff))
                uniform_axis_field_rowwise_left.graph_data.add_edge(axis[0],axis[1])
        for k in range(1,10):
            pt2 = (k*step_size * side_uv_right) + pt
            uv,axis = get_local_avg_axis(axis_field,[pt2],pt2,pia,wm,radius)
            if axis is not None:
                uniform_axis_field.graph_data.add_edge(axis[0],axis[1])
                uniform_axis_field_rowwise.graph_data.add_edge(axis[0],axis[1])
                write_edge_sg(axis,output_path_vM1_axis_field+'_vM1_axis_field_row_{}_{}_{}_{}_{}.am'.\
                              format(row_ind,k,surf_res,voxel_size,cutoff))
                uniform_axis_field_rowwise_right.graph_data.add_edge(axis[0],axis[1])
        if len(uniform_axis_field_rowwise.graph_data.edge_list) > 0:
            uniform_axis_field_rowwise.write_spatial_graph(output_path_vM1_axis_field+\
                                                        '_vM1_axis_field_row_{}_{}_{}_{}.am'.\
                                                           format(row_ind,surf_res,voxel_size,cutoff)) 
        if len(uniform_axis_field_rowwise_left.graph_data.edge_list) > 0:
            uniform_axis_field_rowwise_left.write_spatial_graph(output_path_vM1_axis_field+\
                                                            '_vM1_axis_field_row_{}_left_{}_{}_{}.am'.\
                                                                format(row_ind,surf_res,voxel_size,cutoff)) 
        if len(uniform_axis_field_rowwise_right.graph_data.edge_list) > 0:
            uniform_axis_field_rowwise_right.write_spatial_graph(output_path_vM1_axis_field+\
                                                '_vM1_axis_field_row_{}_right_{}_{}_{}.am'.\
                                                                 format(row_ind,surf_res,voxel_size,cutoff)) 
        row_ind = row_ind + step_direction
    start_ind = start_ind + 1

uniform_axis_field.write_spatial_graph(output_path_vM1_axis_field+'vM1_axis_field_{}.am'.format(hem))  



## Make avg layers

In [ ]:
# Make avg layers
surf_res = 50
df = pd.read_csv(output_root+'Final_Data/Layer_Borders/vM1_Borders.csv',index_col=False)

for hem in ['lhs','rhs']:
    output_path_vM1_axis_field = output_root+'/Final_Data/Average/vM1/'
    exp = 'Uniform_Axis_Field_{}_New/'.format(hem)
    for layer in ['L5B_Border_Per']:#['L1_Border_Per','L2_3_Border_Per','L5A_Border_Per','L5B_Border_Per',]:
        #layer_depth = lb[layer][i]*100
        # mean layer border
        layer_depth_mean = df[layer].mean(axis=0) * 100
        layer_depth_upper = layer_depth_mean - df[layer].std(axis=0) * 100
        layer_depth_lower = layer_depth_mean + df[layer].std(axis=0) * 100
        #print(layer)
        #print(layer_depth_mean)
        #print(layer_depth_upper)
        #print(layer_depth_lower)
        for layer_depth in [layer_depth_upper]:#[layer_depth_mean,layer_depth_upper,layer_depth_lower]:
            i = i +1
            layer_surf = Surface()
            arr = [-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10]
            for j in range(len(arr)):
                #j = arr[k]

                vectors_sg_left = None
                vectors_sg_right = None
                vectors_sg_next_left = None
                vectors_sg_next_right = None
                layer_border_pts_left = []
                layer_border_pts_right = []
                if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                  .format(arr[j],surf_res,400,0.1)) and \
                       os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                      .format(arr[j+1],surf_res,400,0.1)):
                    vectors_sg = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                                   .format(arr[j],surf_res,400,0.1),\
                                                   generic_graph=True)
                    if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                      .format(arr[j],surf_res,400,0.1)):
                        vectors_sg_left = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                                            .format(arr[j],surf_res,400,0.1),\
                                                   generic_graph=True)
                    if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                      .format(arr[j],surf_res,400,0.1)):
                        vectors_sg_right = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                                             .format(arr[j],surf_res,400,0.1),\
                                                   generic_graph=True)

                    vectors_sg_next = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                                        .format(arr[j+1],surf_res,400,0.1),\
                                                   generic_graph=True)

                    if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                      .format(arr[j+1],surf_res,400,0.1)):
                        vectors_sg_next_left = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                                                 .format(arr[j+1],surf_res,400,0.1),\
                                                   generic_graph=True)
                    if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                      .format(arr[j+1],surf_res,400,0.1)):
                        vectors_sg_next_right = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                                                  .format(arr[j+1],surf_res,400,0.1),\
                                                   generic_graph=True)

                    # now fetch layer specific pt
                    if len(vectors_sg.graph_data.edge_list) > 0 and len(vectors_sg_next.graph_data.edge_list) > 0:
                        pts = []
                        left_pts = []
                        right_pts = []
                        layer_border_pts = Vectors().get_layer_pts_for_given_depth(vectors_sg.graph_data.edge_list,layer_depth,\
                                                                                     realtive_depths=True,invert_axes=True,rel_depth_given=True)
                        if vectors_sg_left is not None and len(vectors_sg_left.graph_data.edge_list) > 0:
                            layer_border_pts_left = Vectors().get_layer_pts_for_given_depth(vectors_sg_left.graph_data.edge_list,layer_depth,\
                                                                                     realtive_depths=True,invert_axes=True,rel_depth_given=True)
                        if vectors_sg_right is not None and len(vectors_sg_right.graph_data.edge_list) > 0:
                            layer_border_pts_right = Vectors().get_layer_pts_for_given_depth(vectors_sg_right.graph_data.edge_list,layer_depth,\
                                                                                     realtive_depths=True,invert_axes=True,rel_depth_given=True)

                        layer_border_pts_next = Vectors().get_layer_pts_for_given_depth(vectors_sg_next.graph_data.edge_list,layer_depth,\
                                                                                     realtive_depths=True,invert_axes=True,rel_depth_given=True)
                        if vectors_sg_next_left is not None and len(vectors_sg_next_left.graph_data.edge_list) > 0: 
                            layer_border_pts_next_left = Vectors().get_layer_pts_for_given_depth(vectors_sg_next_left.graph_data.edge_list,layer_depth,\
                                                                                     realtive_depths=True,invert_axes=True,rel_depth_given=True)
                        if vectors_sg_next_right is not None and len(vectors_sg_next_right.graph_data.edge_list) > 0:
                            layer_border_pts_next_right = Vectors().get_layer_pts_for_given_depth(vectors_sg_next_right.graph_data.edge_list,layer_depth,\
                                                                                     realtive_depths=True,invert_axes=True,rel_depth_given=True)

                        for pt in layer_border_pts:
                            pts.append(pt)
                        for pt in layer_border_pts_next:
                            pts.append(pt)

                        for pt in layer_border_pts_left:
                            left_pts.append(pt)
                        for pt in layer_border_pts_next_left:
                            left_pts.append(pt)
                        for pt in layer_border_pts_right:
                            right_pts.append(pt)
                        for pt in layer_border_pts_next_right:
                            right_pts.append(pt)

                        # left side surface
                        #max(len(left_pts))
                        #print(len(layer_border_pts_left),len(layer_border_pts_next_left))
                        for pt_ind in range(max(len(layer_border_pts_left),len(layer_border_pts_next_left))-1):
                            #print(pt_ind)
                            pts = []
                            #pts_next = []
                            if len(layer_border_pts_left)>0:
                                if pt_ind>len(layer_border_pts_left)-2:
                                    pts.append(layer_border_pts_left[len(layer_border_pts_left)-1])
                                else:
                                    pts.append(layer_border_pts_left[pt_ind])
                                    pts.append(layer_border_pts_left[pt_ind+1])
                            if len(layer_border_pts_next_left)>0:
                                if pt_ind>len(layer_border_pts_next_left)-2:
                                    pts.append(layer_border_pts_next_left[len(layer_border_pts_next_left)-1])
                                else:
                                    pts.append(layer_border_pts_next_left[pt_ind])
                                    pts.append(layer_border_pts_next_left[pt_ind+1])
                            #print(pts)
                            #pts_next = [layer_border_pts_left[pt_ind+1],layer_border_pts_left[pt_ind+1]]

                            surf = Surface(pts=pts).create_delunay_surface_2d(return_hull=True,alpha=0,offset=100)

                            layer_surf.append(surf)
                        for pt_ind in range(max(len(layer_border_pts_right),len(layer_border_pts_next_right))-1):
                            #print(pt_ind)
                            pts = []
                            #pts_next = []
                            if len(layer_border_pts_right)>0:
                                if pt_ind>len(layer_border_pts_right)-2:
                                    pts.append(layer_border_pts_right[len(layer_border_pts_right)-1])
                                else:
                                    pts.append(layer_border_pts_right[pt_ind])
                                    pts.append(layer_border_pts_right[pt_ind+1])
                            if len(layer_border_pts_next_right)>0:
                                if pt_ind>len(layer_border_pts_next_right)-2:
                                    pts.append(layer_border_pts_next_right[len(layer_border_pts_next_right)-1])
                                else:
                                    pts.append(layer_border_pts_next_right[pt_ind])
                                    pts.append(layer_border_pts_next_right[pt_ind+1])
                            #print(pts)
                            #pts_next = [layer_border_pts_left[pt_ind+1],layer_border_pts_left[pt_ind+1]]
                            surf = Surface(pts=pts).create_delunay_surface_2d(return_hull=True,alpha=0,offset=100)
                            layer_surf.append(surf)

            vertices = []
            for vert_ind in range(layer_surf.surface.GetNumberOfPoints()):
                vertices.append(layer_surf.surface.GetPoints().GetPoint(vert_ind))

            output_path_neun_layer_borders = output_root+'Final_Data/Average/vM1/'
            exp = 'Avg_'+hem
            # write mean, upper or lower layer border surface
            if layer_depth == layer_depth_mean:
                Landmarks(pts = vertices).write_landmarks( output_path_neun_layer_borders+ exp +'_vM1_'+layer[:-4]+'_{}_mean.landmarksAscii'.format(surf_res))
                layer_surf.write_surface_mesh(output_path_neun_layer_borders+exp +'_vM1_'+layer[:-4]+'_{}_mean.vtk'.format(surf_res))
            elif layer_depth == layer_depth_upper:
                print(layer_depth)
                Landmarks(pts = vertices).write_landmarks( output_path_neun_layer_borders+ exp +'_vM1_'+layer[:-4]+'_{}_upper.landmarksAscii'.format(surf_res))
                layer_surf.write_surface_mesh(output_path_neun_layer_borders+exp +'_vM1_'+layer[:-4]+'_{}_upper.vtk'.format(surf_res))
            else:
                Landmarks(pts = vertices).write_landmarks( output_path_neun_layer_borders+ exp +'_vM1_'+layer[:-4]+'_{}_lower.landmarksAscii'.format(surf_res))
                layer_surf.write_surface_mesh(output_path_neun_layer_borders+exp +'_vM1_'+layer[:-4]+'_{}_lower.vtk'.format(surf_res))



In [ ]:
# rabies layer border
df = pd.read_csv(output_root+'Final_Data/Layers_Rabies/vM1_Borders.csv',index_col=False)
df['L5_Upper_Border'].mean(),df['L5_Lower_Border'].mean()

# Figure 2 - Cellular Organization (Layers)

## Use the following versions: vM1 : NeuN-V1, Rabies-V2, vS1 : NeuN-V2, Rabies-V1


## Find % Rabies in L5

In [ ]:
# find % rabies in L5 V1
df_main = pd.DataFrame()
for exp in ['MG48','MG49','MG50']:
    for hem in ['lhs','rhs']:
        for br in ['vM1',]:
            landmarks = Landmarks(output_root+'Final_Data/Landmarks/Density_Clusters/{}_{}_{}_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii'.format(exp,hem,br))
            landmarks_L5 = Landmarks(output_root+'Final_Data/L5_Rabies_Landmarks/V1/{}_{}_{}_rabies_landmarks_L5.landmarkAscii'.format(exp,hem,br))
            landmarks_L5B = Landmarks(output_root+'Final_Data/L5_Rabies_Landmarks/V1/{}_{}_{}_rabies_landmarks_L5B.landmarkAscii'.format(exp,hem,br))
            
            df = pd.DataFrame()
            df['Exp'] = [exp]
            df['Hem'] = [hem]
            df['BR'] = [br]
            df['total_count'] = [len(landmarks.pts)]
            df['L5_count'] = [len(landmarks_L5.pts)]
            df['L5B_count'] = [len(landmarks_L5B.pts)]
            df['L5_per'] = [len(landmarks_L5.pts)/len(landmarks.pts)]
            df['L5B_per'] = [len(landmarks_L5B.pts)/len(landmarks_L5.pts)]
            
            df_main = df_main.append(df)
df_main.to_csv(output_root+'Final_Data/L5_Rabies_Landmarks/V1/L5_counts.csv')

In [ ]:
v1_borders = pd.read_csv(output_root+'Final_Data/Layer_Borders/vM1_Borders.csv')
v2_borders = pd.read_csv(output_root+'Final_Data/Layer_Borders_V2/vM1_Borders.csv')

## Make local ref frame vM1

In [ ]:
def get_edge_endpt(edge,length):
    uvec = Vectors().get_unit_vec(edge[0],edge[1])
    
    return uvec*length

In [ ]:
# vM1, vS1 and barrels local ref frames
# use pca to obtain the local ref frames
for exp in ['MG48','MG49','MG50']:
    for hem in ['lhs','rhs']:
        for br in ['vM1',]:
            if exp == 'Avg':
                pca0 = AmiraSpatialGraph(output_root + 'Final_Data/Average/{}/Avg_{}_{}_PCA0_Trimmed.am'.format(br,br,hem),generic_graph=True).graph_data.edge_list[0]
                pca1 = AmiraSpatialGraph(output_root + 'Final_Data/Average/{}/Avg_{}_{}_PCA1_Trimmed.am'.format(br,br,hem),generic_graph=True).graph_data.edge_list[0]
                pca2 = AmiraSpatialGraph(output_root + 'Final_Data/Average/{}/Avg_{}_{}_PCA2_Trimmed.am'.format(br,br,hem),generic_graph=True).graph_data.edge_list[0]
            else:
                pca0 = AmiraSpatialGraph(output_root + 'Final_Data/{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_Neg.am'.format(br,exp,hem),generic_graph=True).graph_data.edge_list[0]
                pca1 = AmiraSpatialGraph(output_root + 'Final_Data/{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_Neg.am'.format(br,exp,hem),generic_graph=True).graph_data.edge_list[0]
                pca2 = AmiraSpatialGraph(output_root + 'Final_Data/{}_Ref_Surfaces/{}_{}_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_Neg.am'.format(br,exp,hem),generic_graph=True).graph_data.edge_list[0]
            
            pts = []
            pts.append(pca0[0])
            pts.append(Vectors().create_pt_along_vector_at_given_distance(1,pca1[0],pca1[1]))
            pts.append(Vectors().create_pt_along_vector_at_given_distance(1,pca0[0],pca0[1]))
            pts.append(Vectors().create_pt_along_vector_at_given_distance(1,pca2[0],pca2[1]))
            
            ref_pts = []
            if exp=='MG48' and hem=='rhs':
                ref_pts.append([0,0,0])
                ref_pts.append([-1,0,0])
                ref_pts.append([0,1,0])
                ref_pts.append([0,0,1])
            elif exp=='MG49' and hem=='rhs':
                ref_pts.append([0,0,0])
                ref_pts.append([-1,0,0])
                ref_pts.append([0,-1,0])
                ref_pts.append([0,0,1])
            elif exp=='MG50' and hem=='rhs':
                ref_pts.append([0,0,0])
                ref_pts.append([-1,0,0])
                ref_pts.append([0,-1,0])
                ref_pts.append([0,0,1])
            elif exp=='Avg' and hem == 'lhs':
                ref_pts.append([0,0,0])
                ref_pts.append([-1,0,0])
                ref_pts.append([0,-1,0])
                ref_pts.append([0,0,-1])
            elif exp=='Avg' and hem == 'rhs':
                ref_pts.append([0,0,0])
                ref_pts.append([1,0,0])
                ref_pts.append([0,-1,0])
                ref_pts.append([0,0,-1])
            else:
                ref_pts.append([0,0,0])
                ref_pts.append([1,0,0])
                ref_pts.append([0,1,0])
                ref_pts.append([0,0,1])
            
            landmarks = Landmarks(pts=pts)
            icp,txmat = landmarks.align_landmarks(ref_pts)

            aligner = Alignment(lhs=(hem=='lhs')) 
            if exp == 'Avg':
                aligner.transform_folders(output_root+'Final_Data/Average/',output_root+'Fig_2/{}_Local_Ref_Frames/Average/'.format(br),exp_name=exp+'_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
                aligner.transform_folders(output_root+'Final_Data/Average/vM1/',output_root+'Fig_2/{}_Local_Ref_Frames/Average/vM1/'.format(br),exp_name=exp+'_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
                aligner.transform_folders(output_root+'Final_Data/Average/',output_root+'Fig_2/{}_Local_Ref_Frames/Average/'.format(br),exp_name=exp+'_vM1_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
                aligner.transform_folders(output_root+'Final_Data/Average/vM1/',output_root+'Fig_2/{}_Local_Ref_Frames/Average/vM1/'.format(br),exp_name=exp+'_vM1_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
            else:
                aligner.transform_folders(output_root+'Final_Data/',output_root+'Fig_2/{}_Local_Ref_Frames/'.format(br),exp_name=exp+'_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
                aligner.transform_folders(output_root+'Final_Data/Landmarks/',output_root+'Fig_2/{}_Local_Ref_Frames/Landmarks/'.format(br),exp_name=exp+'_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
            

## Make local ref frame BF

In [ ]:
def get_cylinder(radius,edge):
    center = (np.array(edge[0])+np.array(edge[1]))/2
    height = Vectors().get_vec_length(edge)
    cylinderSource = vtk.vtkCylinderSource()
    cylinderSource.SetCenter(center)
    cylinderSource.SetRadius(radius)
    cylinderSource.SetHeight(height)
    cylinderSource.SetResolution(100)
    cylinderSource.Update()

    pts = []
    ref_pts = []
    pts.append(center)
    end_pt = [center[0],cylinderSource.GetOutput().GetBounds()[2],center[2] ]
    pts.append(end_pt)

    ref_pts.append(center)
    end_pt = Vectors().get_unit_vec(center,edge[1])*height/2 + center
    ref_pts.append(end_pt)
    #Landmarks(pts=pts).write_landmarks(output_path_neun_profiles+'pts')
    #Landmarks(pts=ref_pts).write_landmarks(output_path_neun_profiles+'ref_pts')

    landmarks = Landmarks(pts=pts)
    icp,txmat = landmarks.align_landmarks(ref_pts)

    cyl_surf = Surface(polydata=cylinderSource.GetOutput())
    cyl_surf.apply_icp_transform(icp)
    #print(txmat)

    #Surface(polydata=cylinderSource.GetOutput()).write_surface_mesh(output_path_neun_profiles+'_cylinder.vtk')

    return cyl_surf.surface 

In [ ]:
# Make named barrel axes and individual axis column cylinders
output_path = output_root + 'Final_Data/'
for exp in ['MG49_lhs','MG49_rhs','MG50_lhs','MG50_rhs',]:
    sg = AmiraSpatialGraph(output_path+'Surfaces/{}_barrels_50.am'.format(exp),barrel_projections_present=True)
    for row in (sg.barrels.rows_list):
        for barrel in row.single_barrels_list:
            if len(barrel.barrel_centroid) > 0:
                cyl = get_cylinder(250,[barrel.wm_projection_barrel_centroid,barrel.pia_projection_barrel_centroid])
                cyl_surf = Surface(polydata=cyl).create_delunay_surface_3d(return_hull=True)
                Surface(polydata=cyl_surf).write_surface_mesh(output_path+'Surfaces/'+exp+'_'+barrel.barrel_name+'_column_cylinder.vtk')
                
                # also make barrel cylinders, but use the column axis
                # priject barrel centers onto column axis
                min_edge,min_dist, min_dist_from_wm, min_dist_from_pia, top_pt_on_column_axis = \
                    Vectors().get_nearest_axis_to_pt(barrel.top_barrel_centroid,[[barrel.wm_projection_barrel_centroid,barrel.pia_projection_barrel_centroid]],\
                                                     axis_validation_distance=3000)
                min_edge,min_dist, min_dist_from_wm, min_dist_from_pia, bottom_pt_on_column_axis = \
                    Vectors().get_nearest_axis_to_pt(barrel.bottom_barrel_centroid,[[barrel.wm_projection_barrel_centroid,barrel.pia_projection_barrel_centroid]],\
                                                     axis_validation_distance=3000)
                cyl = get_cylinder(250,[bottom_pt_on_column_axis,top_pt_on_column_axis])
                cyl_surf = Surface(polydata=cyl).create_delunay_surface_3d(return_hull=True)
                Surface(polydata=cyl_surf).write_surface_mesh(output_path+'Surfaces/'+exp+'_'+barrel.barrel_name+'_barrel_cylinder.vtk')

In [ ]:
# apply local ref frame transformation
#input_path = output_root + 'Final_Data/Surfaces/'
output_path = output_root + 'Final_Data/'

# center is D2 center, 
# z axis is D2 axis
# y axis is D2-C2
# x axis is D2-D3
br = 'BF'
for exp in ['MG49','MG50']:
    for hem in ['lhs','rhs']:
        sg = AmiraSpatialGraph(output_path+'Surfaces/{}_{}_barrels_50.am'.format(exp,hem),barrel_projections_present=True,reverse_barrel_direction=True)
        
        pts = []
        pts.append(sg.barrels.d_row.two.barrel_centroid)
        
        #pts.append(Vectors().create_pt_along_vector_at_given_distance(100,sg.barrels.d_row.two.barrel_centroid,sg.barrels.c_row.two.barrel_centroid))
        #pts.append(Vectors().create_pt_along_vector_at_given_distance(100,sg.barrels.d_row.two.barrel_centroid,sg.barrels.d_row.three.barrel_centroid))
        pts.append(Vectors().create_pt_along_vector_at_given_distance(100,sg.barrels.d_row.two.barrel_centroid,sg.barrels.d_row.two.wm_projection_barrel_centroid))

        Landmarks(pts=pts).write_landmarks(output_path+'bla.landmarksAscii')
        ref_pts = []
        ref_pts.append([0,0,0])
        #if hem == 'lhs':
        #    ref_pts.append([-100,0,0])
        #else:
        #    ref_pts.append([100,0,0])
            
        #ref_pts.append([0,-100,0])
        ref_pts.append([0,0,100])
            
        landmarks = Landmarks(pts=pts)
        icp,txmat = landmarks.align_landmarks(ref_pts)

        aligner = Alignment(lhs=(hem=='lhs')) 
        if exp == 'Avg':
            aligner.transform_folders(output_root+'Final_Data/Average/',output_root+'Fig_2/{}_Local_Ref_Frames/Average/'.format(br),exp_name=exp+'_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
            aligner.transform_folders(output_root+'Final_Data/Average/SBF/',output_root+'Fig_2/{}_Local_Ref_Frames/Average/SBF/'.format(br),exp_name=exp+'_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
            aligner.transform_folders(output_root+'Final_Data/Average/',output_root+'Fig_2/{}_Local_Ref_Frames/Average/'.format(br),exp_name=exp+'_vM1_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
            aligner.transform_folders(output_root+'Final_Data/Average/SBF/',output_root+'Fig_2/{}_Local_Ref_Frames/Average/SBF/'.format(br),exp_name=exp+'_vM1_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
        else:
            aligner.transform_folders(output_root+'Final_Data/',output_root+'Fig_2/{}_Local_Ref_Frames/'.format(br),exp_name=exp+'_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
            aligner.transform_folders(output_root+'Final_Data/Landmarks/',output_root+'Fig_2/{}_Local_Ref_Frames/Landmarks/'.format(br),exp_name=exp+'_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
    
        
    

In [ ]:
# apply local ref frame transformation
#input_path = output_root + 'Final_Data/Surfaces/'
#output_path = output_root + 'Final_Data/'

# center is D2 center, 
# z axis is D2 axis
# x axis is D2-C2
# y axis is D2-D3

# project the D2 center (now the origin ) onto c2 column and make this new d2 c2 as y axis
br = 'BF'
for exp in ['MG49','MG50']:
    for hem in ['lhs','rhs']:
        sg = AmiraSpatialGraph(output_root+'Fig_2/{}_Local_Ref_Frames/'.format(br)+'Surfaces/{}_{}_barrels_50.am'.format(exp,hem),barrel_projections_present=True,reverse_barrel_direction=True)
        
        #print(sg.barrels.d_row.two.barrel_centroid)
        #continue
        pts = []
        pts.append(sg.barrels.d_row.two.barrel_centroid)
        
        #pts.append(Vectors().create_pt_along_vector_at_given_distance(100,sg.barrels.d_row.two.barrel_centroid,sg.barrels.c_row.two.barrel_centroid))
        min_edge,min_dist, min_dist_from_wm, min_dist_from_pia, d2_pt_on_c2_column_axis = \
                    Vectors().get_nearest_axis_to_pt(sg.barrels.d_row.two.barrel_centroid,\
                                                     [[sg.barrels.c_row.two.wm_projection_barrel_centroid,\
                                                       sg.barrels.c_row.two.pia_projection_barrel_centroid]],\
                                                     axis_validation_distance=3000)
        pts.append(Vectors().create_pt_along_vector_at_given_distance(100,sg.barrels.d_row.two.barrel_centroid,d2_pt_on_c2_column_axis))
        #pts.append(Vectors().create_pt_along_vector_at_given_distance(100,sg.barrels.d_row.two.barrel_centroid,sg.barrels.d_row.two.wm_projection_barrel_centroid))

        Landmarks(pts=pts).write_landmarks(output_path+'bla.landmarksAscii')
        ref_pts = []
        ref_pts.append(sg.barrels.d_row.two.barrel_centroid)
        #if hem == 'lhs':
        #    ref_pts.append([-100,0,0])
        #else:
        #    ref_pts.append([100,0,0])
            
        ref_pts.append([0,100,0])
        #ref_pts.append([0,0,100])
            
        landmarks = Landmarks(pts=pts)
        icp,txmat = landmarks.align_landmarks(ref_pts)

        aligner = Alignment(lhs=(hem=='lhs')) 
        if exp == 'Avg':
            aligner.transform_folders(output_root+'Final_Data/Average/',output_root+'Fig_2/{}_Local_Ref_Frames/Average/'.format(br),exp_name=exp+'_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
            aligner.transform_folders(output_root+'Final_Data/Average/SBF/',output_root+'Fig_2/{}_Local_Ref_Frames/Average/SBF/'.format(br),exp_name=exp+'_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
            aligner.transform_folders(output_root+'Final_Data/Average/',output_root+'Fig_2/{}_Local_Ref_Frames/Average/'.format(br),exp_name=exp+'_vM1_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
            aligner.transform_folders(output_root+'Final_Data/Average/SBF/',output_root+'Fig_2/{}_Local_Ref_Frames/Average/SBF/'.format(br),exp_name=exp+'_vM1_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
        else:
            aligner.transform_folders(output_root+'Fig_2/{}_Local_Ref_Frames/'.format(br),output_root+'Fig_2/{}_Local_Ref_Frames/'.format(br),exp_name=exp+'_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
            aligner.transform_folders(output_root+'Fig_2/{}_Local_Ref_Frames/Landmarks/'.format(br),output_root+'Fig_2/{}_Local_Ref_Frames/Landmarks/'.format(br),exp_name=exp+'_'+hem,icp=icp,txmat = txmat,hem_specific_alignment=True)
    
        
    

### create layer broders to cover all of BF


In [ ]:
def get_desired_edge_extrapolation(next_pia_pt,next_wm_pt,pia,wm):
    pia_proj_pt,depth_wm = pia.get_vector_intersection_pt\
                (next_pia_pt,next_wm_pt,extrapolation_len=1000)
            
    wm_proj_pt,depth_wm = wm.get_vector_intersection_pt\
                (next_wm_pt,next_pia_pt,extrapolation_len=-1000)
    return [wm_proj_pt,pia_proj_pt]

In [ ]:
lb = pd.read_csv(output_path_neun_layer_borders_v2+'vS1_Borders.csv')
for exp in ['MG49_lhs','MG50_lhs','MG50_rhs',]:
    print(exp)
    lb = pd.read_csv(output_path_neun_layer_borders_v2+'vS1_Borders.csv')
    for surf_res in [50]:#surface_resolutions:
        
        pia = Surface(output_root+'Fig_2/BF_Local_Ref_Frames/Surfaces/{}_pia_open_bottom_voxel_size_{}.vtk'.format(exp,surf_res))
        wm = Surface(output_root+'Fig_2/BF_Local_Ref_Frames/Surfaces/{}_WM_open_bottom_voxel_size_{}.vtk'.format(exp,surf_res))
        
        #vectors_sg = AmiraSpatialGraph(output_path_vS1_axis_field+exp+'_vS1_axis_field_{}.am'.format(surf_res),generic_graph=True)
        sg = AmiraSpatialGraph(output_root+'Fig_2/BF_Local_Ref_Frames/Surfaces/{}_barrels_50.am'.format(exp),barrel_projections_present=True,reverse_barrel_direction=True)
        vectors_sg = AmiraSpatialGraph(generic_graph=True)
        for row in (sg.barrels.rows_list):
            for barrel in row.single_barrels_list:
                if len(barrel.barrel_centroid) > 0:
                    vectors_sg.graph_data.add_edge(barrel.wm_projection_barrel_centroid,barrel.pia_projection_barrel_centroid)
        
        # add one more edge parallel to d2 on y axis on the boundaries for full coverag
        if exp is not 'MG50_lhs':
            d2_pia_pt =  [sg.barrels.d_row.two.pia_projection_barrel_centroid[0],\
                           sg.barrels.d_row.two.pia_projection_barrel_centroid[1],\
                           sg.barrels.d_row.two.pia_projection_barrel_centroid[2]]
            d2_wm_pt =  [sg.barrels.d_row.two.wm_projection_barrel_centroid[0],\
                           sg.barrels.d_row.two.wm_projection_barrel_centroid[1],\
                           sg.barrels.d_row.two.wm_projection_barrel_centroid[2]]
            next_pia_pt = [d2_pia_pt[0],d2_pia_pt[1]-500,d2_pia_pt[2]]
            next_wm_pt = [d2_wm_pt[0],d2_wm_pt[1]-500,d2_wm_pt[2]]
            edge = get_desired_edge_extrapolation(next_pia_pt,next_wm_pt,pia,wm)
            vectors_sg.graph_data.add_edge(edge[0],edge[1])

            next_pia_pt = [d2_pia_pt[0]+500,d2_pia_pt[1]-500,d2_pia_pt[2]]
            next_wm_pt = [d2_wm_pt[0]+500,d2_wm_pt[1]-500,d2_wm_pt[2]]
            edge = get_desired_edge_extrapolation(next_pia_pt,next_wm_pt,pia,wm)
            vectors_sg.graph_data.add_edge(edge[0],edge[1])

            next_pia_pt = [d2_pia_pt[0]+500,d2_pia_pt[1],d2_pia_pt[2]]
            next_wm_pt = [d2_wm_pt[0]+500,d2_wm_pt[1],d2_wm_pt[2]]
            edge = get_desired_edge_extrapolation(next_pia_pt,next_wm_pt,pia,wm)
            vectors_sg.graph_data.add_edge(edge[0],edge[1])
        
        #vectors_sg.write_spatial_graph(output_root+'bla.am')
        for layer in ['L1_Border_Per','L2_3_Border_Per','L4_Border_Per','L5_Border_Per',]:
            layer_depth_mean = lb[lb['Exp_Name']==exp][layer].mean() * 100
            layer_depth_upper = layer_depth_mean - (lb[lb['Exp_Name']==exp][layer].std() * 100)
            layer_depth_lower = layer_depth_mean + (lb[lb['Exp_Name']==exp][layer].std() * 100)
            #print(layer_depth)
            for layer_depth in [layer_depth_mean,layer_depth_upper,layer_depth_lower]:
                layer_border_pts = Vectors().get_layer_pts_for_given_depth(vectors_sg.graph_data.edge_list,layer_depth,\
                                                                                              realtive_depths=True,invert_axes=True,rel_depth_given=True)
                layer_surf = Surface(pts=layer_border_pts).create_delunay_surface_2d(return_hull=True)
                # write mean, upper or lower layer border surface
                if layer_depth == layer_depth_mean:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_root+'Fig_2/BF_Local_Ref_Frames/Layer_Borders/'+ exp +'_vS1_'+layer[:-4]+'_{}_mean.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_root+'Fig_2/BF_Local_Ref_Frames/Layer_Borders/'+exp +'_vS1_'+layer[:-4]+'_{}_mean.vtk'.format(surf_res))
                elif layer_depth == layer_depth_upper:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_root+'Fig_2/BF_Local_Ref_Frames/Layer_Borders/'+ exp +'_vS1_'+layer[:-4]+'_{}_upper.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_root+'Fig_2/BF_Local_Ref_Frames/Layer_Borders/'+exp +'_vS1_'+layer[:-4]+'_{}_upper.vtk'.format(surf_res))
                else:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_root+'Fig_2/BF_Local_Ref_Frames/Layer_Borders/'+ exp +'_vS1_'+layer[:-4]+'_{}_lower.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_root+'Fig_2/BF_Local_Ref_Frames/Layer_Borders/'+exp +'_vS1_'+layer[:-4]+'_{}_lower.vtk'.format(surf_res))

In [ ]:
# create layers for new avg sbf
# for now use MG50_lhs as avg for visualozation of layer borders
lb = pd.read_csv(output_path_neun_layer_borders_v2+'vS1_Borders.csv')
for exp in ['MG50_lhs']:#['MG49_lhs','MG50_lhs','MG50_rhs',]:
    print(exp)
    lb = pd.read_csv(output_path_neun_layer_borders_v2+'vS1_Borders.csv')
    for surf_res in [50]:#surface_resolutions:
        
        pia = Surface(output_root+'Fig_2/BF_Local_Ref_Frames/Surfaces/{}_pia_open_bottom_voxel_size_{}.vtk'.format(exp,surf_res))
        wm = Surface(output_root+'Fig_2/BF_Local_Ref_Frames/Surfaces/{}_WM_open_bottom_voxel_size_{}.vtk'.format(exp,surf_res))
        
        #vectors_sg = AmiraSpatialGraph(output_path_vS1_axis_field+exp+'_vS1_axis_field_{}.am'.format(surf_res),generic_graph=True)
        sg = AmiraSpatialGraph(output_root+'Fig_2/BF_Local_Ref_Frames/Surfaces/{}_barrels_50.am'.format(exp),barrel_projections_present=True,reverse_barrel_direction=True)
        vectors_sg = AmiraSpatialGraph(generic_graph=True)
        for row in (sg.barrels.rows_list):
            for barrel in row.single_barrels_list:
                if len(barrel.barrel_centroid) > 0:
                    vectors_sg.graph_data.add_edge(barrel.wm_projection_barrel_centroid,barrel.pia_projection_barrel_centroid)
        
        # add one more edge parallel to d2 on y axis on the boundaries for full coverag
        if exp is not 'MG50_lhs':
            d2_pia_pt =  [sg.barrels.d_row.two.pia_projection_barrel_centroid[0],\
                           sg.barrels.d_row.two.pia_projection_barrel_centroid[1],\
                           sg.barrels.d_row.two.pia_projection_barrel_centroid[2]]
            d2_wm_pt =  [sg.barrels.d_row.two.wm_projection_barrel_centroid[0],\
                           sg.barrels.d_row.two.wm_projection_barrel_centroid[1],\
                           sg.barrels.d_row.two.wm_projection_barrel_centroid[2]]
            next_pia_pt = [d2_pia_pt[0],d2_pia_pt[1]-500,d2_pia_pt[2]]
            next_wm_pt = [d2_wm_pt[0],d2_wm_pt[1]-500,d2_wm_pt[2]]
            edge = get_desired_edge_extrapolation(next_pia_pt,next_wm_pt,pia,wm)
            vectors_sg.graph_data.add_edge(edge[0],edge[1])

            next_pia_pt = [d2_pia_pt[0]+500,d2_pia_pt[1]-500,d2_pia_pt[2]]
            next_wm_pt = [d2_wm_pt[0]+500,d2_wm_pt[1]-500,d2_wm_pt[2]]
            edge = get_desired_edge_extrapolation(next_pia_pt,next_wm_pt,pia,wm)
            vectors_sg.graph_data.add_edge(edge[0],edge[1])

            next_pia_pt = [d2_pia_pt[0]+500,d2_pia_pt[1],d2_pia_pt[2]]
            next_wm_pt = [d2_wm_pt[0]+500,d2_wm_pt[1],d2_wm_pt[2]]
            edge = get_desired_edge_extrapolation(next_pia_pt,next_wm_pt,pia,wm)
            vectors_sg.graph_data.add_edge(edge[0],edge[1])
        
        #vectors_sg.write_spatial_graph(output_root+'bla.am')
        for layer in ['L1_Border_Per','L2_3_Border_Per','L4_Border_Per','L5_Border_Per',]:
            layer_depth_mean = lb[layer].mean() * 100
            layer_depth_upper = layer_depth_mean - (lb[lb['Exp_Name']==exp][layer].std() * 100)
            layer_depth_lower = layer_depth_mean + (lb[lb['Exp_Name']==exp][layer].std() * 100)
            #print(layer_depth)
            for layer_depth in [layer_depth_mean,layer_depth_upper,layer_depth_lower]:
                layer_border_pts = Vectors().get_layer_pts_for_given_depth(vectors_sg.graph_data.edge_list,layer_depth,\
                                                                                              realtive_depths=True,invert_axes=True,rel_depth_given=True)
                layer_surf = Surface(pts=layer_border_pts).create_delunay_surface_2d(return_hull=True)
                exp = 'Avg'
                # write mean, upper or lower layer border surface
                if layer_depth == layer_depth_mean:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_root+'Fig_2/BF_Local_Ref_Frames/Layer_Borders/'+ exp +'_vS1_'+layer[:-4]+'_{}_mean.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_root+'Fig_2/BF_Local_Ref_Frames/Layer_Borders/'+exp +'_vS1_'+layer[:-4]+'_{}_mean.vtk'.format(surf_res))
                elif layer_depth == layer_depth_upper:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_root+'Fig_2/BF_Local_Ref_Frames/Layer_Borders/'+ exp +'_vS1_'+layer[:-4]+'_{}_upper.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_root+'Fig_2/BF_Local_Ref_Frames/Layer_Borders/'+exp +'_vS1_'+layer[:-4]+'_{}_upper.vtk'.format(surf_res))
                else:
                    Landmarks(pts = layer_border_pts).write_landmarks( output_root+'Fig_2/BF_Local_Ref_Frames/Layer_Borders/'+ exp +'_vS1_'+layer[:-4]+'_{}_lower.landmarksAscii'.format(surf_res))
                    Surface(polydata=layer_surf).write_surface_mesh(output_root+'Fig_2/BF_Local_Ref_Frames/Layer_Borders/'+exp +'_vS1_'+layer[:-4]+'_{}_lower.vtk'.format(surf_res))

## Clip layer borders for visualization

In [ ]:
def get_local_ref_frame_clipping_plane(cutting_thickness=100):
    ''' For the given surface find the y extent and 
    create clipping planes at the end points '''
    
    
    landmarks_center = [0,0,0]
    print(landmarks_center,cutting_thickness)
    
    pt1 = [landmarks_center[0]+cutting_thickness,landmarks_center[1],landmarks_center[2]]
    pt2 = [landmarks_center[0]+cutting_thickness,landmarks_center[1],landmarks_center[2]+1000]
    pt3 = [landmarks_center[0]+cutting_thickness,landmarks_center[1]+1000,landmarks_center[2]]
    plane_front,tri_front = Surface().get_cutting_plane(pt1,pt2,pt3)
    #Surface(polydata=tri_front).write_surface_mesh(output_root+'bla.vtk')
    #Surface(polydata=pia1).write_surface_mesh(output_root+'b1.vtk')
    pt1 = [landmarks_center[0]-cutting_thickness,landmarks_center[1],landmarks_center[2]]
    pt2 = [landmarks_center[0]-cutting_thickness,landmarks_center[1],landmarks_center[2]+1000]
    pt3 = [landmarks_center[0]-cutting_thickness,landmarks_center[1]+1000,landmarks_center[2]]
    plane_back,tri_back = Surface().get_cutting_plane(pt1,pt2,pt3)
    
    return plane_front,plane_back,tri_front,tri_back

In [ ]:
def clip_surfaces_for_layer_borders(input_path,exp,hem,br,output_path,clipping_plane_front,clipping_plane_back):
    
    pia = Surface(input_path+'vM1_Ref_Surfaces/'+exp+'_'+hem+'_'+'vm1_pia_surf_res_50_voxel_size_400_cutoff_0.1.vtk')
    wm = Surface(input_path+'vM1_Ref_Surfaces/'+exp+'_'+hem+'_'+'vm1_wm_surf_res_50_voxel_size_400_cutoff_0.1.vtk')
    
    pia1,wm1 = Surface().clip_surfaces(pia.surface,wm.surface,clipping_plane_front,get_left=True)
    pia2,wm2 = Surface().clip_surfaces(pia1,wm1,clipping_plane_back,get_left=False)
    Surface(polydata=pia2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+'_pia_50.vtk')
    Surface(polydata=wm2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+'_wm_50.vtk')
    
    pia = Surface(input_path+'vM1_Ref_Surfaces/'+exp+'_'+hem+'_'+'vm1_pia_surf_res_300_voxel_size_400_cutoff_0.1.vtk')
    wm = Surface(input_path+'vM1_Ref_Surfaces/'+exp+'_'+hem+'_'+'vm1_wm_surf_res_300_voxel_size_400_cutoff_0.1.vtk')
    
    pia1,wm1 = Surface().clip_surfaces(pia.surface,wm.surface,clipping_plane_front,get_left=True)
    pia2,wm2 = Surface().clip_surfaces(pia1,wm1,clipping_plane_back,get_left=False)
    Surface(polydata=pia2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+'_pia.vtk')
    Surface(polydata=wm2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+'_wm.vtk')

    for layer in ['L1','L2_3','L5A','L5B']:
        l = Surface(input_path+'Layer_Borders/'+exp+'_'+hem+'_'+'vM1_{}_Border_300_mean.vtk'.format(layer))
        l1,l1t = Surface().clip_surfaces(l.surface,l.surface,clipping_plane_front,get_left=True)
        l2,l2t = Surface().clip_surfaces(l1,l1,clipping_plane_back,get_left=False)
        Surface(polydata=l2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+layer+'_mean.vtk')
        
        l = Surface(input_path+'Layer_Borders/'+exp+'_'+hem+'_'+'vM1_{}_Border_50_mean.vtk'.format(layer))
        l1,l1t = Surface().clip_surfaces(l.surface,l.surface,clipping_plane_front,get_left=True)
        l2,l2t = Surface().clip_surfaces(l1,l1,clipping_plane_back,get_left=False)
        Surface(polydata=l2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+layer+'_mean_50.vtk')
        
    neun = Landmarks(input_path+'Landmarks/'+exp+'_'+hem+'_neun_landmarks_vM1_surf_res_50_voxel_size_500_cutoff_0.05.LandmarkAscii')
    surf = Surface()
    surf.append(pia2)
    surf.append(wm2)
    
    clipped_landmarks = neun.get_landmarks_in_bounding_box(surf.surface.GetBounds())
    Landmarks(pts=clipped_landmarks).write_landmarks(output_path+exp+hem+br+'clipped_neun.landmarksAscii')
    
    neun2 = Landmarks(input_path+'Landmarks/'+exp+'_'+hem+'_neun_landmarks_vM1.LandmarkAscii')
    clipped_landmarks2 = neun2.get_landmarks_in_bounding_box(surf.surface.GetBounds())
    Landmarks(pts=clipped_landmarks2).write_landmarks(output_path+exp+hem+br+'clipped_neun2.landmarksAscii')
    
    rabies = Landmarks(input_path+'Landmarks/Density_Clusters/'+exp+'_'+hem+'_vM1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii')
    clipped_rabies = rabies.get_landmarks_in_bounding_box(surf.surface.GetBounds())
    Landmarks(pts=clipped_rabies).write_landmarks(output_path+exp+hem+br+'clipped_rabies.landmarksAscii')

In [ ]:
def clip_surfaces_for_layer_borders_BF(input_path,exp,hem,br,output_path,clipping_plane_front,clipping_plane_back):
    
    pia = Surface(input_path+'Surfaces/'+exp+'_'+hem+'_'+'pia_open_bottom_voxel_size_50.vtk')
    wm = Surface(input_path+'Surfaces/'+exp+'_'+hem+'_'+'WM_open_bottom_voxel_size_50.vtk')
    
    pia1,wm1 = Surface().clip_surfaces(pia.surface,wm.surface,clipping_plane_front,get_left=True)
    pia2,wm2 = Surface().clip_surfaces(pia1,wm1,clipping_plane_back,get_left=False)
    Surface(polydata=pia2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+'_pia_50.vtk')
    Surface(polydata=wm2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+'_wm_50.vtk')
    
    pia = Surface(input_path+'vS1_Ref_Surfaces/'+exp+'_'+hem+'_'+'vs1_pia_surf_res_300_voxel_size_400_cutoff_0.1.vtk')
    wm = Surface(input_path+'vS1_Ref_Surfaces/'+exp+'_'+hem+'_'+'vs1_wm_surf_res_300_voxel_size_400_cutoff_0.1.vtk')
    
    pia1,wm1 = Surface().clip_surfaces(pia.surface,wm.surface,clipping_plane_front,get_left=True)
    pia2,wm2 = Surface().clip_surfaces(pia1,wm1,clipping_plane_back,get_left=False)
    Surface(polydata=pia2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+'_pia.vtk')
    Surface(polydata=wm2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+'_wm.vtk')

    for layer in ['L1','L2_3','L4','L5']:
        l = Surface(input_path+'Layer_Borders/'+exp+'_'+hem+'_'+'vS1_{}_Border_300_mean.vtk'.format(layer))
        l1,l1t = Surface().clip_surfaces(l.surface,l.surface,clipping_plane_front,get_left=True)
        l2,l2t = Surface().clip_surfaces(l1,l1,clipping_plane_back,get_left=False)
        Surface(polydata=l2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+layer+'_mean.vtk')
        
        l = Surface(input_path+'Layer_Borders/'+exp+'_'+hem+'_'+'vS1_{}_Border_50_mean.vtk'.format(layer))
        l1,l1t = Surface().clip_surfaces(l.surface,l.surface,clipping_plane_front,get_left=True)
        l2,l2t = Surface().clip_surfaces(l1,l1,clipping_plane_back,get_left=False)
        Surface(polydata=l2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+layer+'_mean_50.vtk')
        
    neun = Landmarks(input_path+'Landmarks/'+exp+'_'+hem+'_neun_landmarks_vS1.LandmarkAscii')
    surf = Surface()
    surf.append(pia2)
    surf.append(wm2)
    
    clipped_landmarks = neun.get_landmarks_in_bounding_box(surf.surface.GetBounds())
    Landmarks(pts=clipped_landmarks).write_landmarks(output_path+exp+hem+br+'clipped_neun.landmarksAscii')
    
    #neun2 = Landmarks(input_path+'Landmarks/'+exp+'_'+hem+'_neun_landmarks_vM1.LandmarkAscii')
    #clipped_landmarks2 = neun2.get_landmarks_in_bounding_box(surf.surface.GetBounds())
    #Landmarks(pts=clipped_landmarks2).write_landmarks(output_path+exp+hem+br+'clipped_neun2.landmarksAscii')
    
    rabies = Landmarks(input_path+'Landmarks/Density_Clusters/'+exp+'_'+hem+'_vS1_rabies_landmarks_voxel_400_cutoff_0.1.LandmarkAscii')
    clipped_rabies = rabies.get_landmarks_in_bounding_box(surf.surface.GetBounds())
    Landmarks(pts=clipped_rabies).write_landmarks(output_path+exp+hem+br+'clipped_rabies.landmarksAscii')

In [ ]:
def clip_surfaces_for_layer_borders_BF_Avg(input_path,exp,hem,br,output_path,clipping_plane_front,clipping_plane_back):
    exp = 'MG50'
    hem = 'lhs'
    pia = Surface(input_path+'Surfaces/'+exp+'_'+hem+'_'+'pia_open_bottom_voxel_size_50.vtk')
    wm = Surface(input_path+'Surfaces/'+exp+'_'+hem+'_'+'WM_open_bottom_voxel_size_50.vtk')
    
    pia1,wm1 = Surface().clip_surfaces(pia.surface,wm.surface,clipping_plane_front,get_left=True)
    pia2,wm2 = Surface().clip_surfaces(pia1,wm1,clipping_plane_back,get_left=False)
    Surface(polydata=pia2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+'_pia_50.vtk')
    Surface(polydata=wm2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+'_wm_50.vtk')
    
    pia = Surface(input_path+'vS1_Ref_Surfaces/'+exp+'_'+hem+'_'+'vs1_pia_surf_res_300_voxel_size_400_cutoff_0.1.vtk')
    wm = Surface(input_path+'vS1_Ref_Surfaces/'+exp+'_'+hem+'_'+'vs1_wm_surf_res_300_voxel_size_400_cutoff_0.1.vtk')
    
    pia1,wm1 = Surface().clip_surfaces(pia.surface,wm.surface,clipping_plane_front,get_left=True)
    pia2,wm2 = Surface().clip_surfaces(pia1,wm1,clipping_plane_back,get_left=False)
    Surface(polydata=pia2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+'_pia.vtk')
    Surface(polydata=wm2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+'_wm.vtk')

    for layer in ['L1','L2_3','L4','L5']:
        l = Surface(input_path+'Layer_Borders/'+'Avg_vS1_{}_Border_300_mean.vtk'.format(layer))
        l1,l1t = Surface().clip_surfaces(l.surface,l.surface,clipping_plane_front,get_left=True)
        l2,l2t = Surface().clip_surfaces(l1,l1,clipping_plane_back,get_left=False)
        Surface(polydata=l2).write_surface_mesh(output_path+'Avg_'+br+layer+'_mean.vtk')
        
        l = Surface(input_path+'Layer_Borders/'+'Avg_vS1_{}_Border_50_mean.vtk'.format(layer))
        l1,l1t = Surface().clip_surfaces(l.surface,l.surface,clipping_plane_front,get_left=True)
        l2,l2t = Surface().clip_surfaces(l1,l1,clipping_plane_back,get_left=False)
        Surface(polydata=l2).write_surface_mesh(output_path+'Avg_'+br+layer+'_mean_50.vtk')
        
    

In [ ]:
def clip_surfaces_for_layer_borders_avg(input_path,exp,hem,br,output_path,clipping_plane_front,clipping_plane_back):
    
    pia = Surface(input_path+'Average/'+br+'/'+exp+'_'+br+'_'+hem+'_'+'Pia.vtk')
    wm = Surface(input_path+'Average/'+br+'/'+exp+'_'+br+'_'+hem+'_'+'WM.vtk')
    
    pia1,wm1 = Surface().clip_surfaces(pia.surface,wm.surface,clipping_plane_front,get_left=True)
    pia2,wm2 = Surface().clip_surfaces(pia1,wm1,clipping_plane_back,get_left=False)
    Surface(polydata=pia2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+'_pia_50.vtk')
    Surface(polydata=wm2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+'_wm_50.vtk')

    for layer in ['L1','L2_3','L5A','L5B']:
        l = Surface(input_path+'Average/'+br+'/'+exp+'_'+hem+'_'+'vM1_{}_Border_50_mean.vtk'.format(layer))
        l1,l1t = Surface().clip_surfaces(l.surface,l.surface,clipping_plane_front,get_left=True)
        l2,l2t = Surface().clip_surfaces(l1,l1,clipping_plane_back,get_left=False)
        Surface(polydata=l2).write_surface_mesh(output_path+exp+'_'+hem+'_'+br+layer+'_mean.vtk')
        
    

In [ ]:
# clip along X axis 

for br in ['BF',]:
    for exp in ['Avg']:#['MG49','MG50']:
        for hem in ['lhs']:#['lhs','rhs']:
            input_path = output_root+'Fig_2/{}_Local_Ref_Frames/'.format(br)
            output_path = output_root+'Fig_2/{}_Local_Ref_Frames/Clipped/'.format(br)
            pathlib.Path((output_path)).mkdir(exist_ok=True)
            plane_front,plane_back,vm1_tri_front,vm1_tri_back = get_local_ref_frame_clipping_plane(200)
            
            #Surface(polydata=vm1_tri_front).write_surface_mesh(output_root+'vm1_tri_front.vtk')
            #Surface(polydata=vm1_tri_back).write_surface_mesh(output_root+'vm1_tri_back.vtk')
            #Surface(polydata=vs1_tri_front).write_surface_mesh(output_root+'vs1_tri_front.vtk')
            #Surface(polydata=vs1_tri_back).write_surface_mesh(output_root+'vs1_tri_back.vtk')
            if exp == 'Avg':
                if br == 'BF':
                    # have used MG50_lhs instead of avg BF
                    clip_surfaces_for_layer_borders_BF_Avg(input_path,exp,hem,br,output_path,plane_front,plane_back)
                else:
                    clip_surfaces_for_layer_borders_avg(input_path,exp,hem,br,output_path,plane_front,plane_back)
            else:
                clip_surfaces_for_layer_borders_BF(input_path,exp,hem,br,output_path,plane_front,plane_back)
            #print(pia2_rhs.GetNumberOfCells())
            # clip rabies landmarks
            #cortical_rabies_lhs = Landmarks(input_path+'/Landmarks/K-Means/{}_lhs_rabies_kmeans_cluster_{}.landmarksAscii'.format(exp,br))
            #cortical_rabies_rhs = Landmarks(input_path+'/Landmarks/K-Means/{}_rhs_rabies_kmeans_cluster_{}.landmarksAscii'.format(exp,br))

            #print(pia2_lhs.GetBounds())
            #print(len(cortical_rabies_rhs.pts))
            #clipped_landmarks_lhs = cortical_rabies_lhs.get_landmarks_in_bounding_box(pia2_lhs.GetBounds())
            #clipped_landmarks_rhs = cortical_rabies_rhs.get_landmarks_in_bounding_box(pia2_rhs.GetBounds())
            #Landmarks(pts = clipped_landmarks_lhs).write_landmarks(input_path+'/Landmarks/{}_lhs_rabies_cortical_landmarks_clipped.LandmarkAscii'.format(exp))
            #Landmarks(pts = clipped_landmarks_rhs).write_landmarks(input_path+'/Landmarks/{}_rhs_rabies_cortical_landmarks_clipped.LandmarkAscii'.format(exp))
            #print(len(clipped_landmarks_lhs))


## Compare layers with others

### see final_plots

## Variability/ Precision of vM1 layer borders

In [ ]:
# see final_plots

## 2d to 3d layerwise scaling

In [ ]:
# see plots

## Layerwise volumes, surf areas and NeuN Densities and counts

### extend axisfield

In [25]:
# Create uniform vM1 axis field 
voxel_size = 400
cutoff=0.1
surf_res = 50
output_path_vM1 = output_root+'Final_Data/vM1_Ref_Surfaces/'
output_path_vM1_axis_field = output_root+'Final_Data/vM1_Ref_Surfaces_Axis_Fields/'
output_path_surfaces = output_root+'Final_Data/Surfaces/'
axis_field_path = output_root+'Final_Data/Axis_Fields/'
#for voxel_size in [50,100,200,300,400,450,500]:
#    for cutoff in [0,0.05,0.1,0.15,0.2,0.3,0.4,0.5]:
#for surf_res in [400,450,500]:#[50,100,200,300]:#surface_resolutions:
for exp in exp_names:
    print(exp)
    if os.path.exists(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_0.am'.\
                                         format(50,voxel_size,cutoff)) and \
    len(AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_0.am'.\
                                         format(50,voxel_size,cutoff),generic_graph=True).\
       graph_data.edge_list) > 0:
        pia = Surface(output_path_surfaces+exp+'_pia_open_bottom_voxel_size_50.vtk')
        wm = Surface(output_path_surfaces+exp+'_WM_open_bottom_voxel_size_50.vtk')
        print(pia.surface.GetNumberOfCells())
        axis_field  = AmiraSpatialGraph(axis_field_path+exp+'_wm_to_pia_axis_field_voxel_size_{}.am'.\
                                        format(surf_res),generic_graph=True).graph_data.edge_list

        horizontal_axis = [AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_0.am'.\
                                             format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1],\
                    AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_0_Neg.am'.\
                                      format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1]]
        vertical_axis = [AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_2.am'.\
                                           format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1],\
                    AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_2_Neg.am'.\
                                      format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1]]
        side_axis = [AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_1.am'.\
                                       format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1],\
                    AmiraSpatialGraph(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_PCA_1_Neg.am'.\
                                      format(50,voxel_size,cutoff),generic_graph=True).graph_data.edge_list[0][1]]
        center_pt = Landmarks(output_path_vM1+exp+'_surf_res_{}_voxel_size_{}_cutoff_{}_vM1_Centroid.landmarksAscii'.\
                              format(50,voxel_size,cutoff)).pts[0]
        print(horizontal_axis)
        print(vertical_axis)
        print(side_axis)
        
        print(Vectors().get_vec_length(horizontal_axis))
        # make the grid
        grid_pts = []
        step_size = 500
        radius = 800
        uniform_axis_field = AmiraSpatialGraph(generic_graph=True)

        #uniform_axis_field.graph_data.add_edge(vertical_axis[0],vertical_axis[1])
        # project the central axis to wm 
        side_uv_left = Vectors().get_unit_vec(center_pt,side_axis[0])
        side_uv_right = Vectors().get_unit_vec(center_pt,side_axis[1])
        start_ind = 0
        step_direction = -1
        # extend the horzontal axis
        horizontal_axis_begin = Vectors().create_pt_along_vector_at_given_distance(10000,horizontal_axis[1],horizontal_axis[0])
        horizontal_axis_end = Vectors().create_pt_along_vector_at_given_distance(10000,horizontal_axis[0],horizontal_axis[1])

        for end_pt in [horizontal_axis_begin,horizontal_axis_end]:
            step_direction = -1*step_direction
            row_ind = step_direction
            for i in range(start_ind,100):
                print(i)
                uniform_axis_field_rowwise = AmiraSpatialGraph(generic_graph=True)
                uniform_axis_field_rowwise_left = AmiraSpatialGraph(generic_graph=True)
                uniform_axis_field_rowwise_right = AmiraSpatialGraph(generic_graph=True)
                pt = Vectors().create_pt_along_vector_at_given_distance(i*step_size,center_pt,end_pt)
                uv,axis = get_local_avg_axis(axis_field,[pt],pt,pia,wm,radius)
                print(axis)
                if axis is not None:
                    
                    uniform_axis_field.graph_data.add_edge(axis[0],axis[1])
                    uniform_axis_field_rowwise.graph_data.add_edge(axis[0],axis[1])
                    write_edge_sg(axis,output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}_{}.am'.\
                                  format(row_ind,0,surf_res,voxel_size,cutoff))
                    uniform_axis_field_rowwise_left.graph_data.add_edge(axis[0],axis[1])
                    uniform_axis_field_rowwise_right.graph_data.add_edge(axis[0],axis[1])
                    # now move along the minor axis

                    for j in range(1,10):
                        pt1 = (j*step_size * side_uv_left) + pt
                        uv,axis = get_local_avg_axis(axis_field,[pt1],pt1,pia,wm,radius)
                        if axis is not None:
                            uniform_axis_field.graph_data.add_edge(axis[0],axis[1])
                            uniform_axis_field_rowwise.graph_data.add_edge(axis[0],axis[1])
                            write_edge_sg(axis,output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}_{}.am'.\
                                          format(row_ind,-j,surf_res,voxel_size,cutoff))
                            uniform_axis_field_rowwise_left.graph_data.add_edge(axis[0],axis[1])
                    for k in range(1,10):
                        pt2 = (k*step_size * side_uv_right) + pt
                        uv,axis = get_local_avg_axis(axis_field,[pt2],pt2,pia,wm,radius)
                        if axis is not None:
                            uniform_axis_field.graph_data.add_edge(axis[0],axis[1])
                            uniform_axis_field_rowwise.graph_data.add_edge(axis[0],axis[1])
                            write_edge_sg(axis,output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}_{}.am'.\
                                          format(row_ind,k,surf_res,voxel_size,cutoff))
                            uniform_axis_field_rowwise_right.graph_data.add_edge(axis[0],axis[1])
                if len(uniform_axis_field_rowwise.graph_data.edge_list) > 0:
                    uniform_axis_field_rowwise.write_spatial_graph(output_path_vM1_axis_field+exp+\
                                                                '_vM1_axis_field_row_{}_{}_{}_{}.am'.\
                                                                   format(row_ind,surf_res,voxel_size,cutoff)) 
                if len(uniform_axis_field_rowwise_left.graph_data.edge_list) > 0:
                    uniform_axis_field_rowwise_left.write_spatial_graph(output_path_vM1_axis_field+exp+\
                                                                    '_vM1_axis_field_row_{}_left_{}_{}_{}.am'.\
                                                                        format(row_ind,surf_res,voxel_size,cutoff)) 
                if len(uniform_axis_field_rowwise_right.graph_data.edge_list) > 0:
                    uniform_axis_field_rowwise_right.write_spatial_graph(output_path_vM1_axis_field+exp+\
                                                        '_vM1_axis_field_row_{}_right_{}_{}_{}.am'.\
                                                                         format(row_ind,surf_res,voxel_size,cutoff)) 
                row_ind = row_ind + step_direction
            start_ind = start_ind + 1

        uniform_axis_field.write_spatial_graph(output_path_vM1_axis_field+exp+'_vM1_axis_field_surf_res_{}_voxel_size_{}_cutoff_{}.am'.\
                                               format(surf_res,voxel_size,cutoff))  
        

MG49_lhs
343812
[[-8270.27777265728, 808.9733184888316, 2005.097427490402], [-7527.453818349807, 3651.863673739642, -1357.9120074252983]]
[[-8006.609093807453, 878.6624591520947, -908.1553544065059], [-7840.299550566574, 2862.945294115628, 805.9752470549411]]
[[-9246.934941208296, 2476.9607174455277, 168.9879270141787], [-6474.617635453966, 2036.94392347095, 409.3752128471747]]
4465.830881079475
0
[(-7840.2998046875, 2862.943603515625, 805.973876953125), (-8001.30419921875, 941.9552001953125, -853.4795532226562)]


/home/mythreya/conda_envs/anaconda3_4.4_tuebingen_servers/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice.
  if sys.path[0] == '':
/home/mythreya/conda_envs/anaconda3_4.4_tuebingen_servers/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1
[(-7914.15966796875, 2705.142333984375, 976.46533203125), (-8089.3984375, 469.195068359375, 56.67567443847656)]
2
[(-7869.51123046875, 2689.493408203125, 1318.0367431640625), (-8282.2724609375, 346.57025146484375, 705.2295532226562)]
3
[(-7854.5390625, 2846.7822265625, 1629.9869384765625), (-8279.373046875, 556.5122680664062, 1311.98974609375)]
4
[(-7935.5400390625, 2917.651611328125, 1757.32763671875), (-8268.033203125, 689.1182250976562, 1798.18359375)]
5
[(-7883.2060546875, 3037.977783203125, 1913.376708984375), (-8293.48828125, 743.1756591796875, 2161.33935546875)]
6
[(-7838.58935546875, 3168.22021484375, 2090.584228515625), (-8245.6171875, 1088.219970703125, 2426.751220703125)]
7
None
8
None
9
None
10
None
11
None
12
None
13
None
14
None
15
None
16
None
17
None
18
None
19
None
20
None
21
None
22
None
23
None
24
None
25
None
26
None
27
None
28
None
29
None
30
None
31
None
32
None
33
None
34
None
35
None
36
None
37
None
38
None
39
None
40
None
41
None
42
None
43
None
44
None
45
No

None
29
None
30
None
31
None
32
None
33
None
34
None
35
None
36
None
37
None
38
None
39
None
40
None
41
None
42
None
43
None
44
None
45
None
46
None
47
None
48
None
49
None
50
None
51
None
52
None
53
None
54
None
55
None
56
None
57
None
58
None
59
None
60
None
61
None
62
None
63
None
64
None
65
None
66
None
67
None
68
None
69
None
70
None
71
None
72
None
73
None
74
None
75
None
76
None
77
None
78
None
79
None
80
None
81
None
82
None
83
None
84
None
85
None
86
None
87
None
88
None
89
None
90
None
91
None
92
None
93
None
94
None
95
None
96
None
97
None
98
None
99
None
1
[(-7780.94189453125, 3160.882568359375, 554.3860473632812), (-8043.95263671875, 1802.6072998046875, -1539.3756103515625)]
2
[(-7716.7392578125, 3493.763427734375, 415.7358093261719), (-8072.69873046875, 2464.17626953125, -1760.8876953125)]
3
[(-7654.8349609375, 3876.09765625, 276.7513427734375), (-8045.33837890625, 3011.88232421875, -1835.6661376953125)]
4
[(-7634.9130859375, 4322.53466796875, 170.12704467773438), (-7964.

None
86
None
87
None
88
None
89
None
90
None
91
None
92
None
93
None
94
None
95
None
96
None
97
None
98
None
99
None
1
[(-7628.59130859375, 2818.349853515625, 479.7807922363281), (-7649.673828125, 1336.72119140625, -1535.7542724609375)]
2
[(-7522.1142578125, 3119.5205078125, 216.6765899658203), (-7462.43408203125, 2069.041259765625, -1534.6815185546875)]
3
[(-7386.146484375, 3396.764404296875, 56.46390914916992), (-7349.5869140625, 2759.238037109375, -1486.5003662109375)]
4
[(-7298.6982421875, 3806.789306640625, -36.41525650024414), (-7193.990234375, 3103.069580078125, -1798.4688720703125)]
5
[(-7237.2412109375, 4255.6201171875, 77.98997497558594), (-7059.7900390625, 3510.79443359375, -1964.4259033203125)]
6
[(-7250.7822265625, 4627.3408203125, -143.13473510742188), (-6924.85205078125, 3895.428955078125, -2130.869140625)]
7
[(-7051.61181640625, 4989.18310546875, -279.10040283203125), (-6842.041015625, 4334.37646484375, -2148.478759765625)]
8
[(-7201.53515625, 5463.98828125, -418.522583

### extended layer border surfaces

In [35]:
def is_axis_present_in_axis_field(axis,axis_field_sg):
    for edge in axis_field_sg.graph_data.edge_list:
        if edge[0][0]==axis[0][0] and edge[0][1]==axis[0][1] and edge[0][2]==axis[0][2] and \
           edge[1][0]==axis[1][0] and edge[1][1]==axis[1][1] and edge[1][2]==axis[1][2]:
            return True
    print('did not find')
    return False

In [40]:
lb = pd.read_csv(output_root+'Final_Data/Layer_Borders/'+'vM1_Borders.csv')
relative_depths = []
output_path_vM1_axis_field = output_root+'Final_Data/vM1_Ref_Surfaces_Axis_Fields/'
output_path_neun_layer_borders = output_root+'Final_Data/Layer_Borders/'
# Create layer borders
for exp in ['MG49_lhs']:#exp_names:
    for surf_res in ['50']:#surface_resolutions:
        # Read axis field vectors
        whole_vectors_sg = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_surf_res_{}_voxel_size_{}_cutoff_{}.am'.\
                                               format(50,400,0.1),generic_graph=True)
        i = 0
        for layer in ['L1_Border_Per','L2_3_Border_Per','L5A_Border_Per','L5B_Border_Per',]:
            #layer_depth = lb[layer][i]*100
            # mean layer border
            layer_depth_mean = lb[lb['Exp_Name']==exp][layer].mean() * 100
            layer_depth_upper = layer_depth_mean - (lb[lb['Exp_Name']==exp][layer].std() * 100)
            layer_depth_lower = layer_depth_mean + (lb[lb['Exp_Name']==exp][layer].std() * 100)
            #print(layer_depth)
            for layer_depth in [layer_depth_mean,layer_depth_upper,layer_depth_lower]:
                i = i +1
                layer_surf = Surface()
                arr = [-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
                for j in range(len(arr)):
                    #j = arr[k]

                    vectors_sg_left = None
                    vectors_sg_right = None
                    vectors_sg_next_left = None
                    vectors_sg_next_right = None
                    layer_border_pts_left = []
                    layer_border_pts_right = []
                    if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                      .format(arr[j],surf_res,400,0.1)) and \
                           os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                          .format(arr[j+1],surf_res,400,0.1)):
                        vectors_sg = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                                       .format(arr[j],surf_res,400,0.1),\
                                                       generic_graph=True)
                        if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                          .format(arr[j],surf_res,400,0.1)):
                            vectors_sg_left = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                                                .format(arr[j],surf_res,400,0.1),\
                                                       generic_graph=True)
                        if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                          .format(arr[j],surf_res,400,0.1)):
                            vectors_sg_right = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                                                 .format(arr[j],surf_res,400,0.1),\
                                                       generic_graph=True)

                        vectors_sg_next = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_{}_{}_{}.am'\
                                                            .format(arr[j+1],surf_res,400,0.1),\
                                                       generic_graph=True)

                        if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                          .format(arr[j+1],surf_res,400,0.1)):
                            vectors_sg_next_left = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_left_{}_{}_{}.am'\
                                                                     .format(arr[j+1],surf_res,400,0.1),\
                                                       generic_graph=True)
                        if os.path.exists(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                          .format(arr[j+1],surf_res,400,0.1)):
                            vectors_sg_next_right = AmiraSpatialGraph(output_path_vM1_axis_field+exp+'_vM1_axis_field_row_{}_right_{}_{}_{}.am'\
                                                                      .format(arr[j+1],surf_res,400,0.1),\
                                                       generic_graph=True)

                        # now fetch layer specific pt
                        if len(vectors_sg.graph_data.edge_list) > 0 and len(vectors_sg_next.graph_data.edge_list) > 0 :
                            #is_axis_present_in_axis_field(vectors_sg.graph_data.edge_list[0],whole_vectors_sg) and \
                            #is_axis_present_in_axis_field(vectors_sg_next.graph_data.edge_list[0],whole_vectors_sg):
                            pts = []
                            left_pts = []
                            right_pts = []
                            layer_border_pts = Vectors().get_layer_pts_for_given_depth(vectors_sg.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)
                            if vectors_sg_left is not None and len(vectors_sg_left.graph_data.edge_list) > 0:
                                layer_border_pts_left = Vectors().get_layer_pts_for_given_depth(vectors_sg_left.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)
                            if vectors_sg_right is not None and len(vectors_sg_right.graph_data.edge_list) > 0:
                                layer_border_pts_right = Vectors().get_layer_pts_for_given_depth(vectors_sg_right.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)

                            layer_border_pts_next = Vectors().get_layer_pts_for_given_depth(vectors_sg_next.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)
                            if vectors_sg_next_left is not None and len(vectors_sg_next_left.graph_data.edge_list) > 0: 
                                layer_border_pts_next_left = Vectors().get_layer_pts_for_given_depth(vectors_sg_next_left.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)
                            if vectors_sg_next_right is not None and len(vectors_sg_next_right.graph_data.edge_list) > 0:
                                layer_border_pts_next_right = Vectors().get_layer_pts_for_given_depth(vectors_sg_next_right.graph_data.edge_list,layer_depth,\
                                                                                         realtive_depths=True,invert_axes=True,rel_depth_given=True)

                            for pt in layer_border_pts:
                                pts.append(pt)
                            for pt in layer_border_pts_next:
                                pts.append(pt)

                            for pt in layer_border_pts_left:
                                left_pts.append(pt)
                            for pt in layer_border_pts_next_left:
                                left_pts.append(pt)
                            for pt in layer_border_pts_right:
                                right_pts.append(pt)
                            for pt in layer_border_pts_next_right:
                                right_pts.append(pt)

                            # left side surface
                            #max(len(left_pts))
                            #print(len(layer_border_pts_left),len(layer_border_pts_next_left))
                            for pt_ind in range(max(len(layer_border_pts_left),len(layer_border_pts_next_left))-1):
                                #print(pt_ind)
                                pts = []
                                #pts_next = []
                                if len(layer_border_pts_left)>0:
                                    if pt_ind>len(layer_border_pts_left)-2:
                                        pts.append(layer_border_pts_left[len(layer_border_pts_left)-1])
                                    else:
                                        pts.append(layer_border_pts_left[pt_ind])
                                        pts.append(layer_border_pts_left[pt_ind+1])
                                if len(layer_border_pts_next_left)>0:
                                    if pt_ind>len(layer_border_pts_next_left)-2:
                                        pts.append(layer_border_pts_next_left[len(layer_border_pts_next_left)-1])
                                    else:
                                        pts.append(layer_border_pts_next_left[pt_ind])
                                        pts.append(layer_border_pts_next_left[pt_ind+1])
                                #print(pts)
                                #pts_next = [layer_border_pts_left[pt_ind+1],layer_border_pts_left[pt_ind+1]]

                                surf = Surface(pts=pts).create_delunay_surface_2d(return_hull=True,alpha=0,offset=100)

                                layer_surf.append(surf)
                            for pt_ind in range(max(len(layer_border_pts_right),len(layer_border_pts_next_right))-1):
                                #print(pt_ind)
                                pts = []
                                #pts_next = []
                                if len(layer_border_pts_right)>0:
                                    if pt_ind>len(layer_border_pts_right)-2:
                                        pts.append(layer_border_pts_right[len(layer_border_pts_right)-1])
                                    else:
                                        pts.append(layer_border_pts_right[pt_ind])
                                        pts.append(layer_border_pts_right[pt_ind+1])
                                if len(layer_border_pts_next_right)>0:
                                    if pt_ind>len(layer_border_pts_next_right)-2:
                                        pts.append(layer_border_pts_next_right[len(layer_border_pts_next_right)-1])
                                    else:
                                        pts.append(layer_border_pts_next_right[pt_ind])
                                        pts.append(layer_border_pts_next_right[pt_ind+1])
                                #print(pts)
                                #pts_next = [layer_border_pts_left[pt_ind+1],layer_border_pts_left[pt_ind+1]]
                                surf = Surface(pts=pts).create_delunay_surface_2d(return_hull=True,alpha=0,offset=100)
                                layer_surf.append(surf)

                vertices = []
                for vert_ind in range(layer_surf.surface.GetNumberOfPoints()):
                    vertices.append(layer_surf.surface.GetPoints().GetPoint(vert_ind))
                
                # write mean, upper or lower layer border surface
                if layer_depth == layer_depth_mean:
                    Landmarks(pts = vertices).write_landmarks( output_path_neun_layer_borders+ exp +'_vM1_'+layer[:-4]+'_{}_mean_extended.landmarksAscii'.format(surf_res))
                    layer_surf.write_surface_mesh(output_path_neun_layer_borders+exp +'_vM1_'+layer[:-4]+'_{}_mean_extended.vtk'.format(surf_res))
                elif layer_depth == layer_depth_upper:
                    Landmarks(pts = vertices).write_landmarks( output_path_neun_layer_borders+ exp +'_vM1_'+layer[:-4]+'_{}_upper_extended.landmarksAscii'.format(surf_res))
                    layer_surf.write_surface_mesh(output_path_neun_layer_borders+exp +'_vM1_'+layer[:-4]+'_{}_upper_extended.vtk'.format(surf_res))
                else:
                    Landmarks(pts = vertices).write_landmarks( output_path_neun_layer_borders+ exp +'_vM1_'+layer[:-4]+'_{}_lower_extended.landmarksAscii'.format(surf_res))
                    layer_surf.write_surface_mesh(output_path_neun_layer_borders+exp +'_vM1_'+layer[:-4]+'_{}_lower_extended.vtk'.format(surf_res))

                

### create layer border volumes

In [59]:
def get_boundaryline_axes(central_axis,ax1,ax2,ax3,axis_field,pia,wm):
    # find pts parallel to ax1 and parpendicular to ax2 and 3
    # and find the avg of nearby axes to these pts 
    # find the farthest pt on this half and position the axis on it
    
    #center_pt = (np.array(central_axis[0]) + np.array(central_axis[0])) / 2
    # now find the farthest pt and position the avg axis here
    #landmarks_within_half,central_plane = get_trimmed_landmarks_for_given_half(central_axis,ax1[0],ax1[1],ax2[1],ax3[1],landmarks,axis_field,\
    #                                                             pia,wm)
    
    #print('number pts in the half {}'.format(len(landmarks_within_half)))
    #print(central_plane.GetNumberOfCells())
    # find the avg axis 
    pts = []
    pts.append(ax1[1])
    pts.append(np.array(ax2[1]) + np.array(ax1[1])-np.array(ax1[0]))
    pts.append(np.array(ax3[1]) + np.array(ax1[1])-np.array(ax1[0]))
    
    #Landmarks(pts=pts).write_landmarks(output_root+'bounds.landmarksascii')
    uv,avg_axis = get_local_avg_axis(axis_field,pts,ax1[1],pia,wm,1000)
    #uv = Vectors().get_unit_vec(avg_axis[0],avg_axis[1])
    
    
    return avg_axis

def get_boundary_axes(central_axis,axis_field,pcas,neg_pcas,pia,wm,offset=500):
    
    # from pca end pts get 1000um axes
    # right side
    avg_axis_right = get_boundaryline_axes(central_axis,neg_pcas[1],pcas[0],neg_pcas[0],axis_field,pia,wm)
    #write_edge_sg(avg_axis_right,output_root+'avg_axis_right')
    #Landmarks(pts=landmarks_within_half_right).write_landmarks(output_root+'landmarks_within_half_right')
    #Landmarks(pts=[landmarks_within_half_right_max]).write_landmarks(output_root+'landmarks_within_half_right_max')
    #Surface(polydata=central_plane_right).write_surface_mesh(output_root+'central_plane_right.vtk')
    #Landmarks(pts=plane_intersecn_pts_right).write_landmarks(output_root+'plane_intersecn_pts_right')
    
    avg_axis_left = get_boundaryline_axes(central_axis,pcas[1],pcas[0],neg_pcas[0],axis_field,pia,wm)
    #write_edge_sg(avg_axis_left,output_root+'avg_axis_left')
    #Landmarks(pts=landmarks_within_half_left).write_landmarks(output_root+'landmarks_within_half_left')
    #Landmarks(pts=[landmarks_within_half_left_max]).write_landmarks(output_root+'landmarks_within_half_left_max')
    #Surface(polydata=central_plane_left).write_surface_mesh(output_root+'central_plane_left.vtk')
    
    avg_axis_front = get_boundaryline_axes(central_axis,pcas[0],pcas[1],neg_pcas[1],axis_field,pia,wm)
    #write_edge_sg(avg_axis_front,output_root+'avg_axis_front')
    #Landmarks(pts=landmarks_within_half_front).write_landmarks(output_root+'landmarks_within_half_front')
    #Landmarks(pts=[landmarks_within_half_front_max]).write_landmarks(output_root+'landmarks_within_half_front_max')
    #Surface(polydata=central_plane_front).write_surface_mesh(output_root+'central_plane_front.vtk')
    
    avg_axis_back = get_boundaryline_axes(central_axis,neg_pcas[0],pcas[1],neg_pcas[1],axis_field,pia,wm)
    #write_edge_sg(avg_axis_back,output_root+'avg_axis_back')
    #Landmarks(pts=landmarks_within_half_back).write_landmarks(output_root+'landmarks_within_half_back')
    #Landmarks(pts=[landmarks_within_half_back_max]).write_landmarks(output_root+'landmarks_within_half_back_max')
    #Surface(polydata=central_plane_back).write_surface_mesh(output_root+'central_plane_back.vtk')
    
    #print('extreme axes are: ')
    #print(avg_axis_right,avg_axis_left,avg_axis_front,avg_axis_back)
    
    return avg_axis_right,avg_axis_left,avg_axis_front,avg_axis_back

In [60]:
def create_vm1_surface(pia_surf,wm_surf,pcas,neg_pcas,axis_field,central_axis,cutting_order):
    
    # find the edges on the border and find avg axis
    # position the edge on the extreme pt based on its 3d distance from center
    extreme_axis_x,extreme_axis_neg_x,extreme_axis_y,extreme_axis_neg_y = get_boundary_axes(central_axis,axis_field,\
                                                                                            pcas,neg_pcas,pia_surf,wm_surf)
    
    # Clip surfaces usig the border planes
    a,y_delta = Vectors().get_center_and_unit_vec(pcas[0][0],pcas[0][1])
    b,x_delta = Vectors().get_center_and_unit_vec(pcas[1][0],pcas[1][1])
    
    plane_right,tri_right = Surface().get_cutting_plane(extreme_axis_x[0],extreme_axis_x[1],\
                                                        extreme_axis_x[0]+100*y_delta)
    plane_left,tri_left = Surface().get_cutting_plane(extreme_axis_neg_x[0],extreme_axis_neg_x[1],\
                                                        extreme_axis_neg_x[0]+100*y_delta)
    plane_front,tri_top = Surface().get_cutting_plane(extreme_axis_y[0],extreme_axis_y[1],\
                                                        extreme_axis_y[0]+100*x_delta)
    plane_back,tri_bottom = Surface().get_cutting_plane(extreme_axis_neg_y[0],extreme_axis_neg_y[1],\
                                                        extreme_axis_neg_y[0]+100*x_delta)
    Surface(polydata=tri_right).write_surface_mesh(output_root+'tri_right.vtk')
    Surface(polydata=tri_left).write_surface_mesh(output_root+'tri_left.vtk')
    Surface(polydata=tri_top).write_surface_mesh(output_root+'tri_top.vtk')
    Surface(polydata=tri_bottom).write_surface_mesh(output_root+'tri_bottom.vtk')
    
    #Surface(polydata=)
#     vm1_final,vm1_hull,vm1_pia,vm1_wm = Surface().clip_surfaces_using_given_planes(pia_surf,wm_surf,plane_right,plane_left,\
#                                                                            plane_front,plane_back,landmarks=[],\
#                                                                            central_axis=central_axis,\
#                                                                            order_of_cutting=cutting_order)
    return plane_right,plane_left,plane_front,plane_back

In [68]:
# use the vm1 hull to clip the layer surfaces
for exp in ['MG49_lhs']:#exp_names:
    
    if exp == 'MG48_rhs' :
        cutting_order = [True,False,True,False]
    else:
        cutting_order = [True,False,False,True]
                
    vm1_hull = Surface(output_root+'Final_Data/vM1_Ref_Surfaces/{}_vM1_hull_surf_res_50_voxel_size_500_cutoff_0.1.vtk'.format(exp))
    vm1_pia = Surface(output_root+'Final_Data/vM1_Ref_Surfaces/{}_vm1_pia_surf_res_50_voxel_size_500_cutoff_0.1.vtk'.format(exp))
    vm1_wm = Surface(output_root+'Final_Data/vM1_Ref_Surfaces/{}_vm1_wm_surf_res_50_voxel_size_500_cutoff_0.1.vtk'.format(exp))
    
    vm1_axis_field = AmiraSpatialGraph(output_root+'Final_Data/vM1_Ref_Surfaces_Axis_Fields/'+exp+'_vM1_axis_field_surf_res_50_voxel_size_400_cutoff_0.1.am',generic_graph=True)
    
    pia = Surface(output_root+'Final_Data/Surfaces/'+exp+'_pia_open_bottom_voxel_size_50.vtk')
    wm = Surface(output_root+'Final_Data/Surfaces/'+exp+'_WM_open_bottom_voxel_size_50.vtk')
    
    # read pcas
    pca_path = output_root+'Final_Data/vM1_Ref_Surfaces/'
    central_axis = AmiraSpatialGraph(pca_path+exp+'_surf_res_50_voxel_size_400_cutoff_0.1_vM1_Central_Axis.am',generic_graph=True)
    pca_0 = AmiraSpatialGraph(pca_path+exp+'_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0.am',generic_graph=True)
    pca_1 = AmiraSpatialGraph(pca_path+exp+'_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1.am',generic_graph=True)
    pca_2 = AmiraSpatialGraph(pca_path+exp+'_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2.am',generic_graph=True)
    pca_0_neg = AmiraSpatialGraph(pca_path+exp+'_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_0_Neg.am',generic_graph=True)
    pca_1_neg = AmiraSpatialGraph(pca_path+exp+'_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_1_Neg.am',generic_graph=True)
    pca_2_neg = AmiraSpatialGraph(pca_path+exp+'_surf_res_50_voxel_size_400_cutoff_0.1_vM1_PCA_2_Neg.am',generic_graph=True)
    
    pcas = []
    neg_pcas = []
    
    pcas.append(pca_0.graph_data.edge_list[0])
    pcas.append(pca_1.graph_data.edge_list[0])
    pcas.append(pca_2.graph_data.edge_list[0])
    neg_pcas.append(pca_0_neg.graph_data.edge_list[0])
    neg_pcas.append(pca_1_neg.graph_data.edge_list[0])
    neg_pcas.append(pca_2_neg.graph_data.edge_list[0])
    
    # find the cutting planes
    plane_right,plane_left,plane_front,plane_back = create_vm1_surface(pia,wm,pcas,neg_pcas,\
                                                                vm1_axis_field.graph_data.edge_list,\
                                                                central_axis=central_axis,cutting_order=cutting_order)
    
    # using these cutting planes create layer volumes
    l1_surf_extended = Surface(output_root+'Final_Data/Layer_Borders/{}_vM1_L1_Border_50_mean.vtk'.format(exp))
    l23_surf_extended = Surface(output_root+'Final_Data/Layer_Borders/{}_vM1_L2_3_Border_50_mean.vtk'.format(exp))
    l5a_surf_extended = Surface(output_root+'Final_Data/Layer_Borders/{}_vM1_L5A_Border_50_mean.vtk'.format(exp))
    l5b_surf_extended = Surface(output_root+'Final_Data/Layer_Borders/{}_vM1_L5B_Border_50_mean.vtk'.format(exp))
    
    l1_vol,vm1_hull,vm1_pia,vm1_wm = Surface().clip_surfaces_using_given_planes(l1_surf_extended.surface,l23_surf_extended.surface,\
                                                                           plane_back,plane_front,\
                                                                           plane_right,plane_left,landmarks=[],\
                                                                           central_axis=central_axis.graph_data.edge_list[0],\
                                                                           order_of_cutting=cutting_order)
    
    
    #l1_vol = l1_surf_extended.crop_hull(vm1_hull.surface,l1_surf_extended.surface,invert_cell_direction=False)
    #Surface(polydata=l1_vol).write_surface_mesh(output_root+'Final_Data/Layer_Borders/{}_vM1_L1_Volume.vtk'.format(exp))
    #Surface(polydata=l1_vol).write_surface_mesh(output_root+'vM1_L23_Volume.vtk'.format(exp))
    #Surface(polydata=vm1_pia).write_surface_mesh(output_root+'vm1_pia.vtk')
    #Surface(polydata=vm1_wm).write_surface_mesh(output_root+'vm1_wm.vtk')
    #Surface(polydata=vm1_hull).write_surface_mesh(output_root+'vm1_hull.vtk')

[[-7840.299556451794, 2862.945321254615, 805.9752597104607], [-8001.304037427848, 941.957840708407, -853.4772551743004]]
30 30
137 137
[]
[[-7840.299556451794, 2862.945321254615, 805.9752597104607], [-8001.304037427848, 941.957840708407, -853.4772551743004]]
131 131
32 32
(-7985.01611328125, 1136.291015625, -685.601806640625)
[[-7840.299556451794, 2862.945321254615, 805.9752597104607], [-8001.304037427848, 941.957840708407, -853.4772551743004]]
137 137
16 16
(-7985.01611328125, 1136.291015625, -685.601806640625)
[[-7840.299556451794, 2862.945321254615, 805.9752597104607], [-8001.304037427848, 941.957840708407, -853.4772551743004]]
32 37
139 140
[]


### Calculate layer volumes based on their realtive thickness

In [137]:
layer_vols_df = pd.DataFrame()
layer_vols_df['Exp'] = exp_names
volumes = pd.read_csv(output_root+'Fig_1/Rabies_Distribution/Variability/variability.csv')

layer_borders = pd.read_csv(output_root+'Final_Data/Layer_Borders/vM1_Borders.csv')
layer_names = ['L1','L2_3','L5A','L5B']
for layer_ind in range(len(layer_names)):
    vols = []
    for exp in ['MG49','MG50','MG48']:
        for hem in ['lhs','rhs']:
            if layer_ind == 0:
                layer_volume = layer_borders[layer_borders['Exp_Name']==exp+'_'+hem]['{}_Border_Per'.format(layer_names[layer_ind])].mean() * \
                            volumes[(volumes['Exp']==exp) & (volumes['Hem']==hem) & (volumes['BR']=='vM1')]['surf_vol'].values[0]
            else:
                layer_volume = (layer_borders[layer_borders['Exp_Name']==exp+'_'+hem]['{}_Border_Per'.format(layer_names[layer_ind])].mean() \
                                - layer_borders[layer_borders['Exp_Name']==exp+'_'+hem]['{}_Border_Per'.format(layer_names[layer_ind-1])].mean()) * \
                            volumes[(volumes['Exp']==exp) & (volumes['Hem']==hem) & (volumes['BR']=='vM1')]['surf_vol'].values[0]
            vols.append(layer_volume)
            #print(exp,hem,layer,layer_volume)
    layer_vols_df[layer_names[layer_ind]] = vols
    
l6vols = []
for exp in ['MG49','MG50','MG48']:
    for hem in ['lhs','rhs']:
        layer_volume = volumes[(volumes['Exp']==exp) & (volumes['Hem']==hem) & (volumes['BR']=='vM1')]['surf_vol'].values[0]\
                        - layer_vols_df[(layer_vols_df['Exp']==exp+'_'+hem)].sum(axis=1).values[0]
        #print(exp,hem,volumes[(volumes['Exp']==exp) & (volumes['Hem']==hem) & (volumes['BR']=='vM1')]['surf_vol'].values[0])
        #print(layer_vols_df[(layer_vols_df['Exp']==exp+'_'+hem)].sum(axis=1).values[0])
        l6vols.append(layer_volume)
layer_vols_df['L6'] = l6vols

layer_vols_df.to_csv(output_root+'Fig_2/Layer_Volumes.csv')

### Find layer specific cell counts and densities

In [236]:
def get_avg_neun_profile(path,exp):
    
    exp_rows = []
    exp_profiles = []
    for i in range(-10,10):

        file = None
        if i <0:
            if len(glob.glob(path+'_vM1_NeuN_Depth_Profile_Streched_using_'+exp+'_row_{}.csv'.format(i)))>0:
                file = glob.glob(path+'_vM1_NeuN_Depth_Profile_Streched_using_'+exp+'_row_{}.csv'.format(i))[0]
        else:
            if len(glob.glob(path+'_vM1_NeuN_Depth_Profile_Streched_using_'+exp+'_row_{}.csv'.format(i)))>0:
                file = glob.glob(path+'_vM1_NeuN_Depth_Profile_Streched_using_'+exp+'_row_{}.csv'.format(i))[0]
        if file is not None:
            #print(file)
            df = pd.read_csv(file)
            #print(df)
            rows = []
            for j in range(2,len(df.columns)):
                profile_correct_direction_list = list(df[df.columns[j]])
                profile_correct_direction_list.reverse()
                if np.array(profile_correct_direction_list).mean(axis=0) > 25:
                    rows.append(profile_correct_direction_list)
                    exp_profiles.append(profile_correct_direction_list)
                
    return  np.array(exp_profiles).mean(axis=0)

In [237]:
# 2D counts and densities
# Find the avg density profile within the layer
# 2D_count = (dens1+dens2+..+densN)*layer_vol/step_size

df_layer_volumes = pd.read_csv(output_root+'Fig_2/Layer_Volumes.csv')
layer_borders = pd.read_csv(output_root+'Final_Data/Layer_Borders/vM1_Borders.csv')
df_scales = pd.read_csv(output_root+'Fig_2/2D_3D_Scaling_Factors_v2.csv')

layer_columns = ['Exp','Hem','L1','L2_3','L5A','L5B','L6',]

df_2d_counts = pd.DataFrame(columns=layer_columns)
df_3d_counts = pd.DataFrame(columns=layer_columns)
df_3d_densities = pd.DataFrame(columns=layer_columns)


for exp in ['MG48','MG49','MG50']:
    for hem in ['lhs','rhs']:
        # Read the density profile
        avg_profile = get_avg_neun_profile(output_root+'Final_Data/NeuN_Profiles/',exp+'_'+hem)
        
        #print(np.array(avg_profile).mean())
        l1_prof = avg_profile[0:int(len(avg_profile)*layer_borders[layer_borders['Exp_Name']==exp+'_'+hem]['L1_Border_Per'].mean())]
        l1_2d_count = (l1_prof.sum() / len(l1_prof)) *  df_layer_volumes[df_layer_volumes['Exp']==exp+'_'+hem]['L1'].values[0] * 1e-6
        l1_3d_count = l1_2d_count*df_scales['L1_Scale'].values[0]
        l1_3d_density = l1_3d_count/(df_layer_volumes[df_layer_volumes['Exp']==exp+'_'+hem]['L1'].values[0] * 1e-9)
        #print(l1_prof.sum() / len(l1_prof))
        #print(np.array(l1_prof).mean())
        l23_prof = avg_profile[int(len(avg_profile)*layer_borders[layer_borders['Exp_Name']==exp+'_'+hem]['L1_Border_Per'].mean()) :\
                    int(len(avg_profile)*layer_borders[layer_borders['Exp_Name']==exp+'_'+hem]['L2_3_Border_Per'].mean())]
        l23_2d_count = l23_prof.sum() / len(l23_prof) *  df_layer_volumes[df_layer_volumes['Exp']==exp+'_'+hem]['L2_3'].values[0]* 1e-6
        l23_3d_count = l23_2d_count*df_scales['L2_3_Scale'].values[0]
        l23_3d_density = l23_3d_count/(df_layer_volumes[df_layer_volumes['Exp']==exp+'_'+hem]['L2_3'].values[0] * 1e-9)
        #print(l23_prof)
        #print(np.array(l23_prof).mean())
        l5A_prof = avg_profile[int(len(avg_profile)*layer_borders[layer_borders['Exp_Name']==exp+'_'+hem]['L2_3_Border_Per'].mean()) :\
                    int(len(avg_profile)*layer_borders[layer_borders['Exp_Name']==exp+'_'+hem]['L5A_Border_Per'].mean())]
        l5A_2d_count = l5A_prof.sum() / len(l5A_prof) *  df_layer_volumes[df_layer_volumes['Exp']==exp+'_'+hem]['L5A'].values[0]* 1e-6
        l5A_3d_count = l5A_2d_count*df_scales['L5A_Scale'].values[0]
        l5A_3d_density = l5A_3d_count/(df_layer_volumes[df_layer_volumes['Exp']==exp+'_'+hem]['L5A'].values[0] * 1e-9)
        #print(l5A_prof)
        #print(np.array(l5A_prof).mean())
        l5B_prof = avg_profile[int(len(avg_profile)*layer_borders[layer_borders['Exp_Name']==exp+'_'+hem]['L5A_Border_Per'].mean()) :\
                    int(len(avg_profile)*layer_borders[layer_borders['Exp_Name']==exp+'_'+hem]['L5B_Border_Per'].mean())]
        l5B_2d_count = l5B_prof.sum() / len(l5B_prof) *  df_layer_volumes[df_layer_volumes['Exp']==exp+'_'+hem]['L5B'].values[0]* 1e-6
        l5B_3d_count = l5B_2d_count*df_scales['L5B_Scale'].values[0]
        l5B_3d_density = l5B_3d_count/(df_layer_volumes[df_layer_volumes['Exp']==exp+'_'+hem]['L5B'].values[0] * 1e-9)
        #print(l5B_prof)
        #print(np.array(l5B_prof).mean())
        l6_prof = avg_profile[int(len(avg_profile)*layer_borders[layer_borders['Exp_Name']==exp+'_'+hem]['L5B_Border_Per'].mean()) :\
                    len(avg_profile)]
        l6_2d_count = l6_prof.sum() / len(l6_prof) *  df_layer_volumes[df_layer_volumes['Exp']==exp+'_'+hem]['L6'].values[0]* 1e-6
        l6_3d_count = l6_2d_count*df_scales['L6_Scale'].values[0]
        l6_3d_density = l6_3d_count/(df_layer_volumes[df_layer_volumes['Exp']==exp+'_'+hem]['L6'].values[0] * 1e-9)
        #print(l6_prof)
        #print(np.array(l6_prof).mean())
        df_2d_counts = df_2d_counts.append(pd.DataFrame(columns=layer_columns,\
                        data=[[exp,hem,l1_2d_count,l23_2d_count,l5A_2d_count,l5B_2d_count,l6_2d_count]]))
                                           
        df_3d_counts = df_3d_counts.append(pd.DataFrame(columns=layer_columns,\
                        data=[[exp,hem,l1_3d_count,l23_3d_count,l5A_3d_count,l5B_3d_count,l6_3d_count]]))
                                           
        df_3d_densities = df_3d_densities.append(pd.DataFrame(columns=layer_columns,\
                        data=[[exp,hem,l1_3d_density,l23_3d_density,l5A_3d_density,l5B_3d_density,l6_3d_density]]))
        
    
        
        
df_2d_counts.to_csv(output_root+'Fig_2/Layerwise_2D_Cell_Counts.csv')
df_3d_counts.to_csv(output_root+'Fig_2/Layerwise_3D_Cell_Counts.csv')
df_3d_densities.to_csv(output_root+'Fig_2/Layerwise_3D_Densities.csv')

In [239]:
df_scales

,Unnamed: 0,L1_Scale,L2_3_Scale,L5A_Scale,L5B_Scale,L6_Scale
0,0,0.098956,2.262202,3.141904,2.382338,2.007678


# Figure 3 - Global v/s Local Precisions

In [243]:
# create avg vm1 
pca2 = AmiraSpatialGraph(output_root+'Fig_2/vM1_Local_Ref_Frames/Average/vM1/Avg_vM1_lhs_PCA2_Trimmed.am',generic_graph=True)
pca2_neg = AmiraSpatialGraph(output_root+'Fig_2/vM1_Local_Ref_Frames/Average/vM1/Avg_vM1_lhs_PCA_2_Neg_Trimmed.am',generic_graph=True)

In [253]:
edge = [pca2.graph_data.edge_list[0][1],pca2_neg.graph_data.edge_list[0][1]]
radius = 250
#def get_cylinder(radius,edge,get_original=False):
center = (np.array(edge[0])+np.array(edge[1]))/2
height = Vectors().get_vec_length(edge)
cylinderSource = vtk.vtkCylinderSource()
cylinderSource.SetCenter(center)
cylinderSource.SetRadius(radius)
cylinderSource.SetHeight(height)
cylinderSource.SetResolution(100)
cylinderSource.Update()

pts = []
ref_pts = []
pts.append(center)
end_pt = [center[0],cylinderSource.GetOutput().GetBounds()[2],center[2] ]
pts.append(end_pt)

ref_pts.append(center)
end_pt = Vectors().get_unit_vec(center,edge[1])*height/2 + center
ref_pts.append(end_pt)
#Landmarks(pts=pts).write_landmarks(output_path_neun_profiles+'pts')
#Landmarks(pts=ref_pts).write_landmarks(output_path_neun_profiles+'ref_pts')

landmarks = Landmarks(pts=pts)
icp,txmat = landmarks.align_landmarks(ref_pts)

cyl_surf = Surface(polydata=cylinderSource.GetOutput())
cyl_surf.apply_icp_transform(icp)
#print(txmat)

cyl_surf_hull = Surface(polydata=cyl_surf.surface).create_delunay_surface_3d(return_hull=True)

Surface(polydata=cyl_surf_hull).write_surface_mesh(output_root+'Fig_3/central_vm1_cylinder.vtk')

In [ ]:
# 

# Old vs New SBF

In [271]:
# Compare these parameters column heights, barrel heights, top, bottom,
# precision: barrel centers, column centers, orientation
cols = ['Exp','Barrel_Name','Barrel_Height','Barrel_Top','Barrel_Bottom','Barrel_Center','Column_Height',]
df = pd.DataFrame(columns=cols)
for exp in exp_names_tangential:
    sg = AmiraSpatialGraph(output_root+'Final_Data/Surfaces/'+exp+'_barrels_50.am',barrel_projections_present=True)
    for row in sg.barrels.rows_list:
        for barrel in row.single_barrels_list:
            if len(barrel.top_barrel_centroid)>0 and len(barrel.bottom_barrel_centroid)>0 and \
                len(barrel.pia_projection_barrel_centroid)>0 and len(barrel.wm_projection_barrel_centroid)>0:
            
                df_tmp = pd.DataFrame(columns=cols)
                df_tmp['Exp']  = [exp]
                df_tmp['Barrel_Name'] = [barrel.barrel_name]
                df_tmp['Barrel_Height']  = [distance.euclidean(barrel.top_barrel_centroid, barrel.bottom_barrel_centroid)] 
                df_tmp['Barrel_Top']  = [distance.euclidean(barrel.pia_projection_barrel_centroid, barrel.top_barrel_centroid)]
                df_tmp['Barrel_Bottom']  = [distance.euclidean(barrel.pia_projection_barrel_centroid, barrel.bottom_barrel_centroid)]
                df_tmp['Barrel_Center']  = [distance.euclidean(barrel.pia_projection_barrel_centroid, barrel.barrel_centroid)]
                df_tmp['Column_Height']  = [distance.euclidean(barrel.pia_projection_barrel_centroid, barrel.wm_projection_barrel_centroid)]

                df = df.append(df_tmp)
            
df.to_csv(output_root+'Fig_2/barrel_variability.csv')

In [274]:
df['Barrel_Height'].mean(),df['Barrel_Height'].std(),

(152.35663949166354, 197.38083968394773)

In [275]:
df_sbf = pd.read_csv(output_root+'Fig_2/sbf_measurements.csv')

FileNotFoundError: File b'/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/FINAL/Fig_2/sbf_measurements.csv' does not exist

# Tangential Avg Ref Frame

In [ ]:
# Flip and align

In [ ]:
# create avg pia and wm

In [ ]:
# create avg axis field

In [ ]:
# create layers borders

In [258]:
v1_borders = pd.read_csv(output_root+'Final_Data/Layer_Borders/vM1_Borders.csv')

In [259]:
v1_borders.mean(),v1_borders.std()

(Unnamed: 0            0.000000
 L1_Border          -222.093023
 L2_3_Border        -544.186047
 L5A_Border        -1017.571059
 L5B_Border        -2025.968992
 L1_Border_Per         0.085420
 L2_3_Border_Per       0.209302
 L5A_Border_Per        0.391373
 L5B_Border_Per        0.779219
 dtype: float64, Unnamed: 0           0.000000
 L1_Border           55.022479
 L2_3_Border        208.929039
 L5A_Border         152.530612
 L5B_Border         228.967674
 L1_Border_Per        0.021162
 L2_3_Border_Per      0.080357
 L5A_Border_Per       0.058666
 L5B_Border_Per       0.088064
 dtype: float64)